```
MODELS:
1. TabPFN
2. ANN
3. PSO-XGBoost
4. PSO-GBR
5. PSO-CatBoost
6. PSO-AdaBoost
```
Features:
1. Flue Gas CO2 (mol%)
2. Flue Gas Flow (kg/h)
3. Lean Solvent Flow (kg/h)
4. CO2-amine Loading (Lean Solvent Loading)
5. Amine Type
6. Amine Concentration (wt%)
7. Lean Solvent Temperature
8. Reboiler Duty

Targets:
1. Recovery%
2. SEC (kJ/kg)
3. CO2 Purity (mol%)


# **Data cleaning and pre-processing**

## Importing the Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

## Importing the data and cleaning

In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)
# # ----------------------------------------------------------
# df = pd.read_excel("C:\\Users\\Pgshco\\Desktop\\Python ML\\Carbon Capture\\ML codes\\Final Code\\FinalData_Rev03.xlsx")
# # ----------------------------------------------------------
# Convert to CSV and display
df.to_csv("Data.csv", index=True)
print("First 5 rows of data:")
df.head()

In [ ]:
print("Before dropna:", df.shape)
df_cleaned = df.dropna()
print("After dropna:", df_cleaned.shape)
# Check missing data
print(df.isna().sum())

# Drop only rows where 'Amine type' is missing
df_cleaned = df.dropna(subset=['Amine type'])

# Encode
df_cleaned = df_cleaned.replace({
    'Amine type': {'MEA': 1, 'DGA': 2, 'DEA': 3, 'TEA': 4, 'MDEA': 5}
})

# Save
df_cleaned.to_csv('cleaned_data_encoded.csv', index=True)
print("✅ Encoded DataFrame saved as 'cleaned_data_encoded.csv'")

# Reload
data = pd.read_csv('cleaned_data_encoded.csv')
data.head()


In [ ]:
data.describe().transpose()

## Feature Selection (before standardization)

### Heat map

In [ ]:
# Drop the "Unnamed: 0" column from the data
data = data.drop(columns=['Unnamed: 0'])

# Now generate the correlation matrix and heatmap
plt.figure(figsize=(12, 6))
mask = np.triu(np.ones_like(data.corr(), dtype=bool), k=1)
heatmap = sns.heatmap(data.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='RdBu')

# Save the plot as a PNG with 600 dpi
plt.savefig('Figures/correlation_heatmap.png', dpi=600, bbox_inches='tight')

### Box plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

box_color = "#377eb8"
cols = list(data.columns)          # 11 parameters
n = len(cols)

fig = plt.figure(figsize=(16, 10))

# Outer grid: 3 rows, 1 column → each row takes full width
outer = gridspec.GridSpec(3, 1, height_ratios=[1, 1, 1], hspace=0.4)

# Row 1: 4 plots
gs_top = outer[0].subgridspec(1, 4, wspace=0.4)
# Row 2: 4 plots
gs_mid = outer[1].subgridspec(1, 4, wspace=0.4)
# Row 3: 3 centered plots
gs_bot = outer[2].subgridspec(1, 3, wspace=0.4)

idx = 0

# ---- First row (4 plots) ----
for j in range(4):
    if idx >= n:
        break
    ax = fig.add_subplot(gs_top[0, j])
    sns.boxplot(
        y=data[cols[idx]],
        ax=ax,
        color=box_color,
        showmeans=True,
        meanprops={"marker": "x",
                   "markeredgecolor": "black",
                   "markerfacecolor": "black"}
    )
    ax.set_title(cols[idx], fontsize=12)
    ax.set_xticks([])
    idx += 1

# ---- Second row (4 plots) ----
for j in range(4):
    if idx >= n:
        break
    ax = fig.add_subplot(gs_mid[0, j])
    sns.boxplot(
        y=data[cols[idx]],
        ax=ax,
        color=box_color,
        showmeans=True,
        meanprops={"marker": "x",
                   "markeredgecolor": "black",
                   "markerfacecolor": "black"}
    )
    ax.set_title(cols[idx], fontsize=12)
    ax.set_xticks([])
    idx += 1

# ---- Third row (3 centered plots) ----
for j in range(3):
    if idx >= n:
        break
    ax = fig.add_subplot(gs_bot[0, j])
    sns.boxplot(
        y=data[cols[idx]],
        ax=ax,
        color=box_color,
        showmeans=True,
        meanprops={"marker": "x",
                   "markeredgecolor": "black",
                   "markerfacecolor": "black"}
    )
    ax.set_title(cols[idx], fontsize=12)
    ax.set_xticks([])
    idx += 1

plt.savefig('Figures/boxPlot.png', dpi=600, bbox_inches='tight')

plt.show()


### Violin-Box Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
import matplotlib.gridspec as gridspec

# ---- Data ----
numeric_df = data.select_dtypes(include=["float64", "int64"])

# ---- Journal styling ----
sns.set_theme(style="whitegrid", font_scale=1.3, rc={
    "axes.labelsize": 14,
    "axes.titlesize": 16,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "axes.linewidth": 1.2,
    "font.family": "serif",
})

# Colors
box_color   = "#377eb8"   # your chosen blue
violin_col  = "#f4a582"   # light red (RdBu-compatible)

cols = list(numeric_df.columns)
num_features = len(cols)   # e.g. 11

# Figure and outer grid: 3 rows x 1 col
fig = plt.figure(figsize=(16, 12))
outer = gridspec.GridSpec(3, 1, figure=fig, hspace=0.6)

# Row 1: 4 plots
gs_top = outer[0].subgridspec(1, 4, wspace=0.4)
# Row 2: 4 plots
gs_mid = outer[1].subgridspec(1, 4, wspace=0.4)
# Row 3: 3 centered plots (they span full width, so naturally centered)
gs_bot = outer[2].subgridspec(1, 3, wspace=0.6)

idx = 0

# ---- First row (up to 4 features) ----
for j in range(4):
    if idx >= num_features:
        break
    ax = fig.add_subplot(gs_top[0, j])

    sns.violinplot(
        y=numeric_df[cols[idx]],
        ax=ax,
        color=violin_col,
        inner=None,
        cut=0,
        bw_adjust=1.4,
        alpha=0.35,
        linewidth=0.6,
        width=0.6
    )

    sns.boxplot(
        y=numeric_df[cols[idx]],
        ax=ax,
        width=0.22,
        color=box_color,
        showmeans=True,
        meanprops={
            "marker": "D",
            "markeredgecolor": "black",
            "markerfacecolor": "black",
            "markersize": 5
        }
    )

    ax.set_title(cols[idx], fontsize=14)
    ax.set_xticks([])
    ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)
    sns.despine(ax=ax, left=True, bottom=True)

    idx += 1

# ---- Second row (next up to 4 features) ----
for j in range(4):
    if idx >= num_features:
        break
    ax = fig.add_subplot(gs_mid[0, j])

    sns.violinplot(
        y=numeric_df[cols[idx]],
        ax=ax,
        color=violin_col,
        inner=None,
        cut=0,
        bw_adjust=1.4,
        alpha=0.35,
        linewidth=0.6,
        width=0.6
    )

    sns.boxplot(
        y=numeric_df[cols[idx]],
        ax=ax,
        width=0.22,
        color=box_color,
        showmeans=True,
        meanprops={
            "marker": "D",
            "markeredgecolor": "black",
            "markerfacecolor": "black",
            "markersize": 5
        }
    )

    ax.set_title(cols[idx], fontsize=14)
    ax.set_xticks([])
    ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)
    sns.despine(ax=ax, left=True, bottom=True)

    idx += 1

# ---- Third row (3 centered plots) ----
for j in range(3):
    if idx >= num_features:
        break
    ax = fig.add_subplot(gs_bot[0, j])

    sns.violinplot(
        y=numeric_df[cols[idx]],
        ax=ax,
        color=violin_col,
        inner=None,
        cut=0,
        bw_adjust=1.4,
        alpha=0.35,
        linewidth=0.6,
        width=0.6
    )

    sns.boxplot(
        y=numeric_df[cols[idx]],
        ax=ax,
        width=0.22,
        color=box_color,
        showmeans=True,
        meanprops={
            "marker": "D",
            "markeredgecolor": "black",
            "markerfacecolor": "black",
            "markersize": 5
        }
    )

    ax.set_title(cols[idx], fontsize=14)
    ax.set_xticks([])
    ax.yaxis.grid(True, linestyle="--", linewidth=0.7, alpha=0.7)
    sns.despine(ax=ax, left=True, bottom=True)

    idx += 1

plt.savefig("Figures/violin_box_subplots.png", dpi=600, bbox_inches="tight")
plt.show()


### Pair plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("white")

# Unified color matching your palette
point_color = "#377eb8"   # chosen medium blue

# Create pairplot with professional styling
g = sns.pairplot(
    data,
    diag_kind="hist",
    plot_kws={
        "s": 25,                # marker size
        "color": point_color,   # fill color
        "edgecolor": "black",   # outline color
        "linewidth": 0.5,       # thickness of outline
        "alpha": 0.8            # slight transparency
    },
    diag_kws={
        "color": point_color,
        "edgecolor": "black"
    }
)

plt.savefig('Figures/pairPlot.png', dpi=1000, bbox_inches='tight')
plt.show()


## Train/Test dataset, standardization, and evaluation function

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv("cleaned_data_encoded.csv")
train_set , test_set = train_test_split(data, test_size=0.2 ,random_state=40,shuffle=True, stratify=None)

# test data
ts = test_set.copy()

ts_label1 = ts["Recovery%"].copy()
ts_label2 = ts["SEC (kJ/kg)"].copy()
ts_label3 = ts["CO2 Purity (mol%)"].copy()
ts = ts.drop(["Recovery%", "SEC (kJ/kg)", "CO2 Purity (mol%)"],axis=1)

# train data
df = train_set.copy()

df_label1 = df["Recovery%"].copy()           # Only the first target is kept
df_label2 = df["SEC (kJ/kg)"].copy()         # Only the second target is kept
df_label3 = df["CO2 Purity (mol%)"].copy()   # Only the third target is kept
df = df.drop(["Recovery%", "SEC (kJ/kg)", "CO2 Purity (mol%)"],axis=1) # drop targets (Only features are kept)

df.head()

In [ ]:
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# train data
df_scaled = pd.DataFrame(
    scaler.fit_transform(df), # train data
    columns=df.columns,      # keep original names
    index=df.index           # keep original index
)
# test data
ts_scaled = pd.DataFrame(
    scaler.transform(ts), # test
    columns=ts.columns, # keep original names
    index=ts.index
)

df = df_scaled.copy()
ts = ts_scaled.copy()
df.head()

**evaluation metrics**

In [ ]:
from sklearn.metrics import (r2_score, mean_squared_error, mean_absolute_error,
                            mean_absolute_percentage_error, median_absolute_error,
                            explained_variance_score, root_mean_squared_log_error)

def evaluate(true, pred, label="Test", control=None):
    # 1. R2 score
    r2 = r2_score(true, pred)
    # 2. Adjusted R2 score
    if label == "Test":
      n = ts.shape[0]
    elif label == "Train":
      n = df.shape[0]
    elif label == "Total":
      n = ts.shape[0] + df.shape[0]
    else:
      raise ValueError("Invalid label. Use 'Test' or 'Train' or 'Total'.")

    k = df.shape[1] # number of features
    adj_r2 = 1 - (1-r2)*(n-1)/ (n-k-1)
    # 3.mean squared error
    mse = mean_squared_error(true, pred)
    # 4.root mean squared error
    rmse = np.sqrt(mse)
    # 5.Normalized RMSE
    nrmse = rmse / (true.max() - true.min())
    # 6.Standard Deviation of Errors
    std_er = np.std(true - pred)
    # 7.Mean Absolute Error
    mae = mean_absolute_error(true, pred)
    # 8.Mean Absolute Percentage Error
    mape = mean_absolute_percentage_error(true, pred)
    # 9.Median Absolute Error
    medae = median_absolute_error(true, pred)
    # 10.Explained Variance Score
    evs = explained_variance_score(true, pred, force_finite=False)

    # Print the metrics
    print(f"\n--- {label} set metrics ---")
    print(f"R²     : {r2:.4f}")
    print(f"Adj-R² : {adj_r2:.4f}")
    print(f"MSE    : {mse:.6f}")
    print(f"RMSE   : {rmse:.6f}")
    print(f"NRMSE  : {nrmse:.6f}")
    print(f"std_er : {std_er:.6f}")
    print(f"MAE    : {mae:.6f}")
    print(f"MAPE   : {mape:.6f}")
    print(f"MedAE  : {medae:.6f}")
    print(f"EVS    : {evs:.6f}")

    return r2, adj_r2, mse, rmse, nrmse, std_er, mae, mape, medae, evs

# **Target1: Recovery**

## ANN-Recovery

In [ ]:
xtrain = df.copy()
xtest = ts.copy()
# Recovery%
ytrain = df_label1.copy()
ytest = ts_label1.copy()
pd.concat([xtrain, ytrain]).describe().transpose()

**tuning the hyperparameters**

In [ ]:
!pip install keras_tuner

In [ ]:
!pip install tensorflow.keras

In [ ]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import random

# --- Fix seeds for reproducibility ---
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# -- Normalizer ---
normalizer = keras.layers.Normalization()
normalizer.adapt(np.array(xtrain))

def build_model(hp):
    model = keras.Sequential()
    model.add(normalizer)

    # Tune number of hidden layers (1 to 3)
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=8),
            activation='relu'
        ))

    # Output layer
    model.add(layers.Dense(1))

    # Tune learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mean_absolute_error'
    )
    return model

# Instantiate tuner (set seed here too)
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='ann_tuning',
    overwrite=True,  # ensures consistency between runs
    seed=SEED        # ensures deterministic search
)

# Search best hyperparameters
tuner.search(xtrain, ytrain,
             epochs=50,
             validation_split=0.2,
             verbose=1)

# Retrieve best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best number of layers:", best_hps.get('num_layers'))
for i in range(best_hps.get('num_layers')):
    print(f"Units in layer {i}:", best_hps.get(f'units_{i}'))
print("Best learning rate:", best_hps.get('learning_rate'))

# Train best model
best_model.fit(xtrain, ytrain, epochs=100, validation_split=0.2, verbose=1)

In [ ]:
best_model.summary()

In [ ]:
best_model.compile(optimizer = keras.optimizers.Adam(), #keras.optimizers.Adam change if needed
              loss= 'mean_absolute_error')
best_model.fit(xtrain, ytrain, verbose=0, epochs=101)

# predctions
y_pred_test_ann1 = best_model.predict(xtest).flatten()
y_pred_train_ann1 = best_model.predict(xtrain).flatten()

In [ ]:
# Make sure both arrays are 1D
train_vals = y_pred_train_ann1.flatten()
test_vals = y_pred_test_ann1.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_ann1": train_vals,
    "y_pred_test_ann1": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("ann1_predictions.xlsx", sheet_name="ann1", index=False)

print("✅ Saved to 'ann1_predictions.xlsx'.")

In [ ]:
evaluate(ytrain, y_pred_train_ann1, "Train")
evaluate(ytest, y_pred_test_ann1, "Test")

plt.figure(figsize=(6,6))
plt.scatter(ytest, y_pred_test_ann1, alpha=0.7, edgecolor="k")
mn = min(np.min(ytest), np.min(y_pred_test_ann1))
mx = max(np.max(ytest), np.max(y_pred_test_ann1))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Recovery")
plt.ylabel("Predicted Recvery")
plt.title("ANN: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## TabPFN-Recovery

**Installing TabPFN**

In [ ]:
!pip install tabpfn

**Training the model**

In [ ]:
from tabpfn import TabPFNRegressor

X_train = df.copy().values
y_train = df_label1.copy().values

X_test = ts.copy().values
y_test = ts_label1.copy().values # Recovery

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train, y_train)

# Predict on the test set
y_pred_test_tab1 = regressor.predict(X_test)
y_pred_train_tab1 = regressor.predict(X_train)

**Saving the predictions to an Excel file**

In [ ]:
# Make sure both arrays are 1D
train_vals = y_pred_train_tab1.flatten()
test_vals = y_pred_test_tab1.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_tab1": train_vals,
    "y_pred_test_tab1": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("tab1_predictions.xlsx", sheet_name="tab1", index=False)

print("✅ Saved to 'tab1_predictions.xlsx'.")

**Plot and metrics**

In [ ]:
evaluate(y_train, y_pred_train_tab1, "Train")
evaluate(y_test, y_pred_test_tab1, "Test")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_tab1, alpha=0.7, edgecolor="k")
mn = min(np.min(y_test), np.min(y_pred_test_tab1))
mx = max(np.max(y_test), np.max(y_pred_test_tab1))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.title("TabPFN: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-XGBoost-Recovery

In [ ]:
!pip install pyswarms

In [ ]:
"""
PSO-tuned XGBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, max_depth, n_estimators, subsample, colsample_bytree, gamma
"""
from xgboost import XGBRegressor
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Load data and prepare
# ---------------------------

y_train = df_label1.copy().values # Recovery
X_train = df.copy().values        # features (scaled)

y_test = ts_label1.copy().values
X_test = ts.copy().values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# We'll tune 6 parameters:
# idx 0 -> learning_rate in [0.001, 0.3]
# idx 1 -> max_depth in [1, 10] (int)
# idx 2 -> n_estimators in [50, 1000] (int)
# idx 3 -> subsample in [0.3, 1.0]
# idx 4 -> colsample_bytree in [0.3, 1.0]
# idx 5 -> gamma in [0.0, 5.0]

# bounds arrays for pyswarms
lower_bounds = np.array([0.001, 1, 50, 0.3, 0.3, 0.0])
upper_bounds = np.array([0.3,   10, 1000, 1.0, 1.0, 5.0])

def _params_from_particle(particle):
    """Map a single particle vector to XGBoost params (with casts)."""
    learning_rate = float(particle[0])
    max_depth = int(np.round(particle[1]))
    max_depth = max(1, max_depth)
    n_estimators = int(np.round(particle[2]))
    n_estimators = max(1, n_estimators)
    subsample = float(particle[3])
    colsample_bytree = float(particle[4])
    gamma = float(particle[5])

    params = {
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "n_estimators": n_estimators,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "gamma": gamma,
        "random_state": RANDOM_STATE,
        "verbosity": 0,
        "objective": "reg:squarederror",
        # 'tree_method':'hist' could speed up on large data (optional)
    }
    return params

def fitness_function(particles):
    """
    particles: array shape (n_particles, dims)
    returns: array shape (n_particles,) of CV MSE (we minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)
    # We use 3-fold CV on training set and return mean MSE
    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = XGBRegressor(**params)
            # negative MSE returned -> convert to positive MSE
            cv_scores = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error", n_jobs=1)
            mse = -np.mean(cv_scores)
            # small regularization to prefer simpler models: add tiny penalty for large n_estimators / depth
            penalty = 1e-6 * (params["n_estimators"] * params["max_depth"])
            scores[i] = mse + penalty
        except Exception as e:
            # something went wrong for this particle: give a large penalty
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
# PSO settings
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}

# dimension equals number of hyperparams
dimensions = 6
# initialize optimizer
optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
    # ftol, etc. could be set; keep defaults
)

print("Starting PSO optimization (this may take some minutes depending on data size)...")
# run optimization: choose moderate iterations to balance runtime vs search
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final XGBoost with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = XGBRegressor(**best_params)
final_model.fit(X_train, y_train)

In [ ]:
y_pred_test_xg1 = final_model.predict(X_test)    # "xg" is the model and "1" is the target which is "Recovery%"
y_pred_train_xg1 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_xg1.flatten()
test_vals = y_pred_test_xg1.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_xg1": train_vals,
    "y_pred_test_xg1": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("xg1_predictions.xlsx", sheet_name="xg1", index=False)

print("✅ Saved to 'xg1_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_xg1, "Train")
evaluate(y_test, y_pred_test_xg1, "Test")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_xg1, alpha=0.7, edgecolor="k")
mn = min(np.min(y_test), np.min(y_pred_test_xg1))
mx = max(np.max(y_test), np.max(y_pred_test_xg1))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.title("PSO-XGBoost: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-GBR-Recovery

In [ ]:
!pip install pyswarms

In [ ]:
y_train_gbr1 = df_label1.values
y_test_gbr1 = ts_label1.values
X_train_gbr1 = df.values
X_test_gbr1 = ts.values

In [ ]:
"""
PSO-tuned Gradient Boosting Regression (GBR)
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, max_depth, n_estimators, subsample,
         min_samples_split, min_samples_leaf, max_features
"""
from sklearn.ensemble import GradientBoostingRegressor
from pyswarms.single import GlobalBestPSO
from sklearn.model_selection import cross_val_score
import numpy as np
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Your prepared data
# ---------------------------
# Assumes these already exist exactly like in your code:
# X_train, y_train, X_test, y_test

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# We'll tune 7 parameters:
# idx 0 -> learning_rate      in [0.005, 0.30]        (float)
# idx 1 -> max_depth          in [1, 8]               (int)
# idx 2 -> n_estimators       in [50, 1000]           (int)
# idx 3 -> subsample          in [0.50, 1.00]         (float)
# idx 4 -> min_samples_split  in [2, 20]              (int)
# idx 5 -> min_samples_leaf   in [1, 10]              (int)
# idx 6 -> max_features_code  in [0.20, 1.00]         (float) -> maps to None if >=0.99, else fraction

# bounds arrays for pyswarms
lower_bounds = np.array([0.005, 1,   50, 0.50,  2,  1, 0.20])
upper_bounds = np.array([0.300, 8, 1000, 1.00, 20, 10, 1.00])

def _params_from_particle(particle):
    """Map a single particle vector to GBR params (with casts and clipping)."""
    learning_rate = float(np.clip(particle[0], lower_bounds[0], upper_bounds[0]))
    max_depth = int(np.round(np.clip(particle[1], lower_bounds[1], upper_bounds[1])))
    n_estimators = int(np.round(np.clip(particle[2], lower_bounds[2], upper_bounds[2])))
    subsample = float(np.clip(particle[3], lower_bounds[3], upper_bounds[3]))
    min_samples_split = int(np.round(np.clip(particle[4], lower_bounds[4], upper_bounds[4])))
    min_samples_leaf  = int(np.round(np.clip(particle[5], lower_bounds[5], upper_bounds[5])))

    # max_features mapping: treat near-1.0 as None (i.e., use all features)
    max_features_code = float(np.clip(particle[6], lower_bounds[6], upper_bounds[6]))
    max_features = None if max_features_code >= 0.99 else float(max_features_code)

    params = {
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "n_estimators": n_estimators,
        "subsample": subsample,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "max_features": max_features,
        "random_state": RANDOM_STATE,
    }
    return params

def fitness_function(particles):
    """
    particles: array shape (n_particles, dims)
    returns: array shape (n_particles,) of CV MSE (we minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = GradientBoostingRegressor(**params)
            # negative MSE returned -> convert to positive MSE
            cv_scores = cross_val_score(
                model, X_train, y_train,
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)

            # tiny regularization to discourage overly large trees / ensembles
            penalty = 1e-6 * (params["n_estimators"] * params["max_depth"])
            scores[i] = mse + penalty
        except Exception:
            # any failure gets a large penalty
            scores[i] = 1e10

    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 7

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),

)

print("Starting PSO optimization for GBR (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final GBR with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = GradientBoostingRegressor(**best_params)
final_model.fit(X_train_gbr1, y_train_gbr1)

In [ ]:
y_pred_test_gbr1 = final_model.predict(X_test_gbr1)
y_pred_train_gbr1 = final_model.predict(X_train_gbr1)

# Make sure both arrays are 1D
train_vals = y_pred_train_gbr1.flatten()
test_vals = y_pred_test_gbr1.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_gbr1": train_vals,
    "y_pred_test_gbr1": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("gbr1_predictions.xlsx", sheet_name="gbr1", index=False)

print("✅ Saved to 'gbr1_predictions.xlsx'.")

In [ ]:
evaluate(df_label1, y_pred_train_gbr1, "Train")
evaluate(ts_label1, y_pred_test_gbr1, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(ts_label1, y_pred_test_gbr1, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([ts_label1.min(), ts_label1.max()], [ts_label1.min(), ts_label1.max()], 'r--', lw=2)
plt.title("GBR: Predicted vs Actual (Test Set)")
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-CatBoost-Recovery

In [ ]:
!pip install pyswarms

In [ ]:
!pip install catboost

In [ ]:
"""
PSO-tuned CatBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, depth, iterations, subsample, rsm, l2_leaf_reg
"""
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Data (same as your XGB block)
# ---------------------------
y_train = df_label1.copy().values  # Recovery
X_train = df.copy().values         # features (scaled)

y_test  = ts_label1.copy().values
X_test  = ts.copy().values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# dims:
# 0 -> learning_rate      [0.001, 0.3]
# 1 -> depth              [1, 10] (int)
# 2 -> iterations         [50, 1000] (int)
# 3 -> subsample          [0.3, 1.0]
# 4 -> rsm                [0.3, 1.0]  (feature subsampling)
# 5 -> l2_leaf_reg        [0.0, 10.0]

lower_bounds = np.array([0.001, 1,   50, 0.3, 0.3, 0.0])
upper_bounds = np.array([0.300, 10, 1000, 1.0, 1.0,10.0])

def _params_from_particle(p):
    """Map a particle to CatBoost params (cast + clip)."""
    learning_rate = float(np.clip(p[0], lower_bounds[0], upper_bounds[0]))
    depth         = int(np.round(np.clip(p[1], lower_bounds[1], upper_bounds[1])))
    iterations    = int(np.round(np.clip(p[2], lower_bounds[2], upper_bounds[2])))
    subsample     = float(np.clip(p[3], lower_bounds[3], upper_bounds[3]))
    rsm           = float(np.clip(p[4], lower_bounds[4], upper_bounds[4]))
    l2_leaf_reg   = float(np.clip(p[5], lower_bounds[5], upper_bounds[5]))

    params = {
        "learning_rate": learning_rate,
        "depth": depth,
        "iterations": iterations,
        "subsample": subsample,
        "rsm": rsm,
        "l2_leaf_reg": l2_leaf_reg,
        "loss_function": "RMSE",
        "random_seed": RANDOM_STATE,
        "verbose": False,
        "allow_writing_files": False,
        "bootstrap_type": "Bernoulli",   # enables 'subsample'
        "task_type": "CPU"
    }
    return params

def fitness_function(particles):
    """
    particles: (n_particles, dims)
    returns:   (n_particles,) mean CV MSE (to minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = CatBoostRegressor(**params)
            # cross_val_score returns neg MSE -> take positive MSE
            cv_scores = cross_val_score(
                model, X_train, y_train,
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)
            # tiny complexity penalty (iterations * depth)
            penalty = 1e-6 * (params["iterations"] * params["depth"])
            scores[i] = mse + penalty
        except Exception:
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 6

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
)

print("Starting PSO optimization for CatBoost (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final CatBoost with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = CatBoostRegressor(**best_params)
final_model.fit(X_train, y_train, verbose=False)

In [ ]:
print("CatBoostRegressor(", end="")
for i, (k, v) in enumerate(final_model.get_params().items()):
    comma = "," if i < len(final_model.get_params()) - 1 else ""
    print(f"{k}={repr(v)}{comma}")
print(")")

In [ ]:
y_pred_test_cb1 = final_model.predict(X_test)
y_pred_train_cb1 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_cb1.flatten()
test_vals = y_pred_test_cb1.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_cb1": train_vals,
    "y_pred_test_cb1": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("cb1_predictions.xlsx", sheet_name="cb1", index=False)

print("✅ Saved to 'cb1_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_cb1, "Train")
evaluate(y_test, y_pred_test_cb1, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_cb1, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("CatBoost: Predicted vs Actual (Test Set)")
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-AdaBoost-Recovery

In [ ]:
!pip install pyswarms

In [ ]:
"""
PSO-tuned AdaBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, n_estimators, max_depth (of base tree),
         min_samples_leaf (of base tree), loss
"""
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Data (same pattern as yours)
# ---------------------------
y_train = df_label1.copy().values  # e.g., Recovery
X_train = df.copy().values
y_test  = ts_label1.copy().values
X_test  = ts.copy().values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# dims:
# 0 -> learning_rate       [0.005, 1.0]
# 1 -> n_estimators        [50, 1000] (int)
# 2 -> max_depth           [1, 10]    (int) for base DecisionTree
# 3 -> min_samples_leaf    [1, 20]    (int) for base DecisionTree
# 4 -> loss_code           [0, 2]     (int) -> 0:'linear', 1:'square', 2:'exponential'

lower_bounds = np.array([0.005,   50,  1,  1, 0])
upper_bounds = np.array([1.000, 1000, 10, 20, 2])

_LOSSES = ['linear', 'square', 'exponential']

def _params_from_particle(p):
    """Map a particle to AdaBoost params (cast + clip)."""
    learning_rate    = float(np.clip(p[0], lower_bounds[0], upper_bounds[0]))
    n_estimators     = int(np.round(np.clip(p[1], lower_bounds[1], upper_bounds[1])))
    max_depth        = int(np.round(np.clip(p[2], lower_bounds[2], upper_bounds[2])))
    min_samples_leaf = int(np.round(np.clip(p[3], lower_bounds[3], upper_bounds[3])))
    loss_code        = int(np.round(np.clip(p[4], lower_bounds[4], upper_bounds[4])))
    loss             = _LOSSES[loss_code]

    # Base learner
    base_tree = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=RANDOM_STATE
    )

    params = {
        "estimator": base_tree,          # sklearn >= 1.2
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "loss": loss,
        "random_state": RANDOM_STATE
    }
    return params, dict(
        learning_rate=learning_rate, n_estimators=n_estimators,
        max_depth=max_depth, min_samples_leaf=min_samples_leaf, loss=loss
    )

def fitness_function(particles):
    """
    particles: (n_particles, dims)
    returns:   (n_particles,) mean CV MSE (to minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params, flat = _params_from_particle(p)
        try:
            model = AdaBoostRegressor(**params)
            # cross_val_score returns neg MSE -> convert to +MSE
            cv_scores = cross_val_score(
                model, X_train, y_train.ravel(),
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)
            # tiny complexity penalty to prefer simpler ensembles
            penalty = 1e-6 * (flat["n_estimators"] * flat["max_depth"])
            scores[i] = mse + penalty
        except Exception:
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 5

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
)

print("Starting PSO optimization for AdaBoost (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final AdaBoost with best params
# ---------------------------
best_params, best_flat = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_flat.items():
    print(f"  {k}: {v}")

final_model = AdaBoostRegressor(**best_params)
final_model.fit(X_train, y_train.ravel())

In [ ]:
print("AdaBoostRegressor(", end="")
for i, (k, v) in enumerate(final_model.get_params().items()):
    comma = "," if i < len(final_model.get_params()) - 1 else ""
    print(f"{k}={repr(v)}{comma}")
print(")")

In [ ]:
y_pred_test_adb1 = final_model.predict(X_test)
y_pred_train_adb1 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_adb1.flatten()
test_vals = y_pred_test_adb1.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_adb1": train_vals,
    "y_pred_test_adb1": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("adb1_predictions.xlsx", sheet_name="adb1", index=False)

print("✅ Saved to 'adb1_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_adb1, "Train")
evaluate(y_test, y_pred_test_adb1, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_adb1, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("AdaBoost: Predicted vs Actual (Test Set)")
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.grid(True)
plt.tight_layout()
plt.show()

# **Target2: SEC**

## ANN-SEC

In [ ]:
xtrain_ann2 = df.copy()
xtest_ann2 = ts.copy()
# SEC
ytrain_ann2 = df_label2.copy()
ytest_ann2 = ts_label2.copy()
pd.concat([xtrain_ann2, ytrain_ann2]).describe().transpose()

**tuning the hyperparameters**

In [ ]:
!pip install keras_tuner

In [ ]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import random

# --- Fix seeds for reproducibility ---
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# -- Normalizer ---
normalizer = keras.layers.Normalization()
normalizer.adapt(np.array(xtrain_ann2))

def build_model(hp):
    model = keras.Sequential()
    model.add(normalizer)

    # Tune number of hidden layers (1 to 3)
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=8),
            activation='relu'
        ))

    # Output layer
    model.add(layers.Dense(1))

    # Tune learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mean_absolute_error'
    )
    return model

# Instantiate tuner (set seed here too)
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='ann_tuning',
    overwrite=True,  # ensures consistency between runs
    seed=SEED        # ensures deterministic search
)

# Search best hyperparameters
tuner.search(xtrain_ann2, ytrain_ann2,
             epochs=50,
             validation_split=0.2,
             verbose=1)

# Retrieve best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best number of layers:", best_hps.get('num_layers'))
for i in range(best_hps.get('num_layers')):
    print(f"Units in layer {i}:", best_hps.get(f'units_{i}'))
print("Best learning rate:", best_hps.get('learning_rate'))

# Train best model
best_model.fit(xtrain_ann2, ytrain_ann2, epochs=100, validation_split=0.2, verbose=1)

In [ ]:
best_model.summary()

In [ ]:
best_model.compile(optimizer = keras.optimizers.Adam(),
              loss= 'mean_absolute_error')
best_model.fit(xtrain_ann2, ytrain_ann2, verbose=0, epochs=101)

In [ ]:
y_pred_test_ann2 = best_model.predict(xtest_ann2).flatten()
y_pred_train_ann2 = best_model.predict(xtrain_ann2).flatten()

# Make sure both arrays are 1D
train_vals = y_pred_train_ann2.flatten()
test_vals = y_pred_test_ann2.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_ann2": train_vals,
    "y_pred_test_ann2": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("ann2_predictions.xlsx", sheet_name="ann2", index=False)

print("✅ Saved to 'ann2_predictions.xlsx'.")

In [ ]:
evaluate(ytrain_ann2, y_pred_train_ann2, "Train")
evaluate(ytest_ann2, y_pred_test_ann2, "Test")

plt.figure(figsize=(6,6))
plt.scatter(ytest_ann2, y_pred_test_ann2, alpha=0.7, edgecolor="k")
mn = min(np.min(ytest_ann2), np.min(y_pred_test_ann2))
mx = max(np.max(ytest_ann2), np.max(y_pred_test_ann2))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.title("ANN: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## TabPFN-SEC

In [ ]:
!pip install tabpfn

In [ ]:
from tabpfn import TabPFNRegressor

X_train_tab2 = df.copy()
y_train_tab2 = df_label2.copy()

X_test_tab2 = ts.copy()
y_test_tab2 = ts_label2.copy()

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train_tab2, y_train_tab2)

# Predict on the test/train set
y_pred_test_tab2 = regressor.predict(X_test_tab2)
y_pred_train_tab2 = regressor.predict(X_train_tab2)

In [ ]:
# Make sure both arrays are 1D
train_vals = y_pred_train_tab2.flatten()
test_vals = y_pred_test_tab2.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_tab2": train_vals,
    "y_pred_test_tab2": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("tab2_predictions.xlsx", sheet_name="tab2", index=False)

print("✅ Saved to 'tab2_predictions.xlsx'.")

In [ ]:
evaluate(y_train_tab2, y_pred_train_tab2, "Train")
evaluate(y_test_tab2, y_pred_test_tab2, "Test")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_test_tab2, y_pred_test_tab2, alpha=0.7, edgecolor="k")
mn = min(np.min(y_test_tab2), np.min(y_pred_test_tab2))
mx = max(np.max(y_test_tab2), np.max(y_pred_test_tab2))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.title("TabPFN: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-XGBoost-SEC

In [ ]:
!pip install pyswarms

In [ ]:
"""
PSO-tuned XGBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, max_depth, n_estimators, subsample, colsample_bytree, gamma
"""

from xgboost import XGBRegressor
from pyswarms.single import GlobalBestPSO
import warnings


warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Load data and prepare
# ---------------------------

y_train_xg2 = df_label2.copy() # SEC
X_train_xg2 = df.copy()        # features (scaled)

y_test_xg2 = ts_label2.copy()
X_test_xg2 = ts.copy()

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# We'll tune 6 parameters:
# idx 0 -> learning_rate in [0.001, 0.3]
# idx 1 -> max_depth in [1, 10] (int)
# idx 2 -> n_estimators in [50, 1000] (int)
# idx 3 -> subsample in [0.3, 1.0]
# idx 4 -> colsample_bytree in [0.3, 1.0]
# idx 5 -> gamma in [0.0, 5.0]

# bounds arrays for pyswarms
lower_bounds = np.array([0.001, 1, 50, 0.3, 0.3, 0.0])
upper_bounds = np.array([0.3,   10, 1000, 1.0, 1.0, 5.0])

def _params_from_particle(particle):
    """Map a single particle vector to XGBoost params (with casts)."""
    learning_rate = float(particle[0])
    max_depth = int(np.round(particle[1]))
    max_depth = max(1, max_depth)
    n_estimators = int(np.round(particle[2]))
    n_estimators = max(1, n_estimators)
    subsample = float(particle[3])
    colsample_bytree = float(particle[4])
    gamma = float(particle[5])

    params = {
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "n_estimators": n_estimators,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "gamma": gamma,
        "random_state": RANDOM_STATE,
        "verbosity": 0,
        "objective": "reg:squarederror",
        # 'tree_method':'hist' could speed up on large data (optional)
    }
    return params

def fitness_function(particles):
    """
    particles: array shape (n_particles, dims)
    returns: array shape (n_particles,) of CV MSE (we minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)
    # We use 3-fold CV on training set and return mean MSE
    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = XGBRegressor(**params)
            # negative MSE returned -> convert to positive MSE
            cv_scores = cross_val_score(model, X_train_xg2, y_train_xg2, cv=3, scoring="neg_mean_squared_error", n_jobs=1)
            mse = -np.mean(cv_scores)
            # small regularization to prefer simpler models: add tiny penalty for large n_estimators / depth
            penalty = 1e-6 * (params["n_estimators"] * params["max_depth"])
            scores[i] = mse + penalty
        except Exception as e:
            # something went wrong for this particle: give a large penalty
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
# PSO settings
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}

# dimension equals number of hyperparams
dimensions = 6
# initialize optimizer
optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
    # ftol, etc. could be set; keep defaults
)

print("Starting PSO optimization (this may take some minutes depending on data size)...")
# run optimization: choose moderate iterations to balance runtime vs search
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final XGBoost with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = XGBRegressor(**best_params)
final_model.fit(X_train_xg2, y_train_xg2)


In [ ]:
y_pred_test_xg2 = final_model.predict(X_test_xg2)    # "xg" is the model and "2" is the targer which is "SEC"
y_pred_train_xg2 = final_model.predict(X_train_xg2)

# Make sure both arrays are 1D
train_vals = y_pred_train_xg2.flatten()
test_vals = y_pred_test_xg2.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_xg2": train_vals,
    "y_pred_test_xg2": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("xg2_predictions.xlsx", sheet_name="xg2", index=False)

print("✅ Saved to 'xg2_predictions.xlsx'.")

In [ ]:
evaluate(y_train_xg2, y_pred_train_xg2, "Train")
evaluate(y_test_xg2, y_pred_test_xg2, "Test")

plt.figure(figsize=(6,6))
plt.scatter(y_test_xg2, y_pred_test_xg2, alpha=0.7, edgecolor="k")
mn = min(np.min(y_test_xg2), np.min(y_pred_test_xg2))
mx = max(np.max(y_test_xg2), np.max(y_pred_test_xg2))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.title("PSO-XGBoost: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-GBR-SEC

In [ ]:
y_train = df_label2.copy()
y_test = ts_label2.copy()
X_train = df.copy()
X_test = ts.copy()

In [ ]:
!pip install pyswarms

In [ ]:
"""
PSO-tuned Gradient Boosting Regression (GBR)
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, max_depth, n_estimators, subsample,
         min_samples_split, min_samples_leaf, max_features
"""
from sklearn.ensemble import GradientBoostingRegressor
from pyswarms.single import GlobalBestPSO
from sklearn.model_selection import cross_val_score
import numpy as np
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Your prepared data
# ---------------------------
# Assumes these already exist exactly like in your code:
# X_train, y_train, X_test, y_test

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# We'll tune 7 parameters:
# idx 0 -> learning_rate      in [0.005, 0.30]        (float)
# idx 1 -> max_depth          in [1, 8]               (int)
# idx 2 -> n_estimators       in [50, 1000]           (int)
# idx 3 -> subsample          in [0.50, 1.00]         (float)
# idx 4 -> min_samples_split  in [2, 20]              (int)
# idx 5 -> min_samples_leaf   in [1, 10]              (int)
# idx 6 -> max_features_code  in [0.20, 1.00]         (float) -> maps to None if >=0.99, else fraction

# bounds arrays for pyswarms
lower_bounds = np.array([0.005, 1,   50, 0.50,  2,  1, 0.20])
upper_bounds = np.array([0.300, 8, 1000, 1.00, 20, 10, 1.00])

def _params_from_particle(particle):
    """Map a single particle vector to GBR params (with casts and clipping)."""
    learning_rate = float(np.clip(particle[0], lower_bounds[0], upper_bounds[0]))
    max_depth = int(np.round(np.clip(particle[1], lower_bounds[1], upper_bounds[1])))
    n_estimators = int(np.round(np.clip(particle[2], lower_bounds[2], upper_bounds[2])))
    subsample = float(np.clip(particle[3], lower_bounds[3], upper_bounds[3]))
    min_samples_split = int(np.round(np.clip(particle[4], lower_bounds[4], upper_bounds[4])))
    min_samples_leaf  = int(np.round(np.clip(particle[5], lower_bounds[5], upper_bounds[5])))

    # max_features mapping: treat near-1.0 as None (i.e., use all features)
    max_features_code = float(np.clip(particle[6], lower_bounds[6], upper_bounds[6]))
    max_features = None if max_features_code >= 0.99 else float(max_features_code)

    params = {
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "n_estimators": n_estimators,
        "subsample": subsample,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "max_features": max_features,
        "random_state": RANDOM_STATE,
    }
    return params

def fitness_function(particles):
    """
    particles: array shape (n_particles, dims)
    returns: array shape (n_particles,) of CV MSE (we minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = GradientBoostingRegressor(**params)
            # negative MSE returned -> convert to positive MSE
            cv_scores = cross_val_score(
                model, X_train, y_train,
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)

            # tiny regularization to discourage overly large trees / ensembles
            penalty = 1e-6 * (params["n_estimators"] * params["max_depth"])
            scores[i] = mse + penalty
        except Exception:
            # any failure gets a large penalty
            scores[i] = 1e10

    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 7

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),

)

print("Starting PSO optimization for GBR (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final GBR with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = GradientBoostingRegressor(**best_params)
final_model.fit(X_train, y_train)

In [ ]:
y_pred_test_gbr2 = final_model.predict(X_test)
y_pred_train_gbr2 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_gbr2.flatten()
test_vals = y_pred_test_gbr2.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_gbr2": train_vals,
    "y_pred_test_gbr2": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("gbr2_predictions.xlsx", sheet_name="gbr2", index=False)

print("✅ Saved to 'gbr2_predictions.xlsx'.")

In [ ]:
evaluate(df_label2, y_pred_train_gbr2, "Train")
evaluate(ts_label2, y_pred_test_gbr2, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(ts_label2, y_pred_test_gbr2, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([ts_label2.min(), ts_label2.max()], [ts_label2.min(), ts_label2.max()], 'r--', lw=2)
plt.title("GBR: Predicted vs Actual (Test Set)")
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-CatBoost-SEC

In [ ]:
!pip install pyswarms

In [ ]:
!pip install catboost

In [ ]:
"""
PSO-tuned CatBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, depth, iterations, subsample, rsm, l2_leaf_reg
"""
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Data (same as your XGB block)
# ---------------------------
y_train = df_label2.copy().values  # Recovery
X_train = df.copy().values         # features (scaled)

y_test  = ts_label2.copy().values
X_test  = ts.copy().values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# dims:
# 0 -> learning_rate      [0.001, 0.3]
# 1 -> depth              [1, 10] (int)
# 2 -> iterations         [50, 1000] (int)
# 3 -> subsample          [0.3, 1.0]
# 4 -> rsm                [0.3, 1.0]  (feature subsampling)
# 5 -> l2_leaf_reg        [0.0, 10.0]

lower_bounds = np.array([0.001, 1,   50, 0.3, 0.3, 0.0])
upper_bounds = np.array([0.300, 10, 1000, 1.0, 1.0,10.0])

def _params_from_particle(p):
    """Map a particle to CatBoost params (cast + clip)."""
    learning_rate = float(np.clip(p[0], lower_bounds[0], upper_bounds[0]))
    depth         = int(np.round(np.clip(p[1], lower_bounds[1], upper_bounds[1])))
    iterations    = int(np.round(np.clip(p[2], lower_bounds[2], upper_bounds[2])))
    subsample     = float(np.clip(p[3], lower_bounds[3], upper_bounds[3]))
    rsm           = float(np.clip(p[4], lower_bounds[4], upper_bounds[4]))
    l2_leaf_reg   = float(np.clip(p[5], lower_bounds[5], upper_bounds[5]))

    params = {
        "learning_rate": learning_rate,
        "depth": depth,
        "iterations": iterations,
        "subsample": subsample,
        "rsm": rsm,
        "l2_leaf_reg": l2_leaf_reg,
        "loss_function": "RMSE",
        "random_seed": RANDOM_STATE,
        "verbose": False,
        "allow_writing_files": False,
        "bootstrap_type": "Bernoulli",
        "task_type": "GPU"
    }
    return params

def fitness_function(particles):
    """
    particles: (n_particles, dims)
    returns:   (n_particles,) mean CV MSE (to minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = CatBoostRegressor(**params)
            # cross_val_score returns neg MSE -> take positive MSE
            cv_scores = cross_val_score(
                model, X_train, y_train,
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)
            # tiny complexity penalty (iterations * depth)
            penalty = 1e-6 * (params["iterations"] * params["depth"])
            scores[i] = mse + penalty
        except Exception:
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 6

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
)

print("Starting PSO optimization for CatBoost (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final CatBoost with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = CatBoostRegressor(**best_params)
# Force CPU right before training to avoid pairwise error
final_model.set_params(task_type="CPU")   # <--- add this line
final_model.fit(X_train, y_train, verbose=False)

In [ ]:
print("CatBoostRegressor(", end="")
for i, (k, v) in enumerate(final_model.get_params().items()):
    comma = "," if i < len(final_model.get_params()) - 1 else ""
    print(f"{k}={repr(v)}{comma}")
print(")")

In [ ]:
y_pred_test_cb2 = final_model.predict(X_test)
y_pred_train_cb2 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_cb2.flatten()
test_vals = y_pred_test_cb2.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_cb2": train_vals,
    "y_pred_test_cb2": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("cb2_predictions.xlsx", sheet_name="cb2", index=False)

print("✅ Saved to 'cb2_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_cb2, "Train")
evaluate(y_test, y_pred_test_cb2, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_cb2, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("CatBoost: Predicted vs Actual (Test Set)")
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-AdaBoost-SEC

In [ ]:
!pip install pyswarms

In [ ]:
"""
PSO-tuned AdaBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, n_estimators, max_depth (of base tree),
         min_samples_leaf (of base tree), loss
"""
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Data (same pattern as yours)
# ---------------------------
y_train = df_label2.copy().values  # e.g., Recovery
X_train = df.copy().values
y_test  = ts_label2.copy().values
X_test  = ts.copy().values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# dims:
# 0 -> learning_rate       [0.005, 1.0]
# 1 -> n_estimators        [50, 1000] (int)
# 2 -> max_depth           [1, 10]    (int) for base DecisionTree
# 3 -> min_samples_leaf    [1, 20]    (int) for base DecisionTree
# 4 -> loss_code           [0, 2]     (int) -> 0:'linear', 1:'square', 2:'exponential'

lower_bounds = np.array([0.005,   50,  1,  1, 0])
upper_bounds = np.array([1.000, 1000, 10, 20, 2])

_LOSSES = ['linear', 'square', 'exponential']

def _params_from_particle(p):
    """Map a particle to AdaBoost params (cast + clip)."""
    learning_rate    = float(np.clip(p[0], lower_bounds[0], upper_bounds[0]))
    n_estimators     = int(np.round(np.clip(p[1], lower_bounds[1], upper_bounds[1])))
    max_depth        = int(np.round(np.clip(p[2], lower_bounds[2], upper_bounds[2])))
    min_samples_leaf = int(np.round(np.clip(p[3], lower_bounds[3], upper_bounds[3])))
    loss_code        = int(np.round(np.clip(p[4], lower_bounds[4], upper_bounds[4])))
    loss             = _LOSSES[loss_code]

    # Base learner
    base_tree = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=RANDOM_STATE
    )

    params = {
        "estimator": base_tree,          # sklearn >= 1.2
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "loss": loss,
        "random_state": RANDOM_STATE
    }
    return params, dict(
        learning_rate=learning_rate, n_estimators=n_estimators,
        max_depth=max_depth, min_samples_leaf=min_samples_leaf, loss=loss
    )

def fitness_function(particles):
    """
    particles: (n_particles, dims)
    returns:   (n_particles,) mean CV MSE (to minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params, flat = _params_from_particle(p)
        try:
            model = AdaBoostRegressor(**params)
            # cross_val_score returns neg MSE -> convert to +MSE
            cv_scores = cross_val_score(
                model, X_train, y_train.ravel(),
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)
            # tiny complexity penalty to prefer simpler ensembles
            penalty = 1e-6 * (flat["n_estimators"] * flat["max_depth"])
            scores[i] = mse + penalty
        except Exception:
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 5

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
)

print("Starting PSO optimization for AdaBoost (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=20, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final AdaBoost with best params
# ---------------------------
best_params, best_flat = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_flat.items():
    print(f"  {k}: {v}")

final_model = AdaBoostRegressor(**best_params)
final_model.fit(X_train, y_train.ravel())

In [ ]:
print("AdaBoostRegressor(", end="")
for i, (k, v) in enumerate(final_model.get_params().items()):
    comma = "," if i < len(final_model.get_params()) - 1 else ""
    print(f"{k}={repr(v)}{comma}")
print(")")

In [ ]:
y_pred_test_adb2 = final_model.predict(X_test)
y_pred_train_adb2 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_adb2.flatten()
test_vals = y_pred_test_adb2.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_adb2": train_vals,
    "y_pred_test_adb2": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("adb2_predictions.xlsx", sheet_name="adb2", index=False)

print("✅ Saved to 'adb2_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_adb2, "Train")
evaluate(y_test, y_pred_test_adb2, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_adb2, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("AdaBoost: Predicted vs Actual (Test Set)")
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.grid(True)
plt.tight_layout()
plt.show()

# **Target3: Purity**

## ANN-Purity

In [ ]:
xtrain = df.copy()
xtest = ts.copy()
# Recovery%
ytrain = df_label3.copy()
ytest = ts_label3.copy()
pd.concat([xtrain, ytrain]).describe().transpose()

**tuning the hyperparameters**

In [ ]:
!pip install keras_tuner

In [ ]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import random

# --- Fix seeds for reproducibility ---
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# -- Normalizer ---
normalizer = keras.layers.Normalization()
normalizer.adapt(np.array(xtrain))

def build_model(hp):
    model = keras.Sequential()
    model.add(normalizer)

    # Tune number of hidden layers (1 to 3)
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=8),
            activation='relu'
        ))

    # Output layer
    model.add(layers.Dense(1))

    # Tune learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mean_absolute_error'
    )
    return model

# Instantiate tuner (set seed here too)
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='ann_tuning',
    overwrite=True,  # ensures consistency between runs
    seed=SEED        # ensures deterministic search
)

# Search best hyperparameters
tuner.search(xtrain, ytrain,
             epochs=50,
             validation_split=0.2,
             verbose=1)

# Retrieve best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best number of layers:", best_hps.get('num_layers'))
for i in range(best_hps.get('num_layers')):
    print(f"Units in layer {i}:", best_hps.get(f'units_{i}'))
print("Best learning rate:", best_hps.get('learning_rate'))

# Train best model
best_model.fit(xtrain, ytrain, epochs=100, validation_split=0.2, verbose=1)

In [ ]:
best_model.summary()

In [ ]:
best_model.compile(optimizer = keras.optimizers.Adam(), #keras.optimizers.Adam change if needed
              loss= 'mean_absolute_error')
best_model.fit(xtrain, ytrain, verbose=0, epochs=101)

y_pred_test_ann3 = best_model.predict(xtest).flatten()
y_pred_train_ann3 = best_model.predict(xtrain).flatten()

In [ ]:
# Make sure both arrays are 1D
train_vals = y_pred_train_ann3.flatten()
test_vals = y_pred_test_ann3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_ann3": train_vals,
    "y_pred_test_ann3": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("ann3_predictions.xlsx", sheet_name="ann3", index=False)

print("✅ Saved to 'ann3_predictions.xlsx'.")

In [ ]:
evaluate(ytrain, y_pred_train_ann3, "Train")
evaluate(ytest, y_pred_test_ann3, "Test")

plt.figure(figsize=(6,6))
plt.scatter(ytest, y_pred_test_ann3, alpha=0.7, edgecolor="k")
mn = min(np.min(ytest), np.min(y_pred_test_ann3))
mx = max(np.max(ytest), np.max(y_pred_test_ann3))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.title("ANN: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## TabPFN-Purity

In [ ]:
!pip install tabpfn

In [ ]:
from tabpfn import TabPFNRegressor

X_train = df.values
y_train = df_label3.values

X_test = ts.values
y_test = ts_label3.values

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train, y_train)

y_pred_test_tab3 = regressor.predict(X_test)
y_pred_train_tab3 = regressor.predict(X_train)

In [ ]:
# Make sure both arrays are 1D
train_vals = y_pred_train_tab3.flatten()
test_vals = y_pred_test_tab3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_tab3": train_vals,
    "y_pred_test_tab3": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("tab3_predictions.xlsx", sheet_name="tab3", index=False)

print("✅ Saved to 'tab3_predictions.xlsx'.")

In [ ]:
# Predict on the test set
evaluate(y_train, y_pred_train_tab3, "Train")
evaluate(y_test, y_pred_test_tab3, "Test")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_tab3, alpha=0.7, edgecolor="k")
mn = min(np.min(y_test), np.min(y_pred_test_tab3))
mx = max(np.max(y_test), np.max(y_pred_test_tab3))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=3)
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.title("TabPFN: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-XGBoost-Purity

In [ ]:
!pip install pyswarms

In [ ]:
from xgboost import XGBRegressor
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Load data and prepare
# ---------------------------

y_train = df_label3.values # Purity
X_train = df.values        # features (scaled)

y_test = ts_label3.values
X_test = ts.values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# We'll tune 6 parameters:
# idx 0 -> learning_rate in [0.001, 0.3]
# idx 1 -> max_depth in [1, 10] (int)
# idx 2 -> n_estimators in [50, 1000] (int)
# idx 3 -> subsample in [0.3, 1.0]
# idx 4 -> colsample_bytree in [0.3, 1.0]
# idx 5 -> gamma in [0.0, 5.0]

# bounds arrays for pyswarms
lower_bounds = np.array([0.001, 1, 50, 0.3, 0.3, 0.0])
upper_bounds = np.array([0.3,   10, 1000, 1.0, 1.0, 5.0])

def _params_from_particle(particle):
    """Map a single particle vector to XGBoost params (with casts)."""
    learning_rate = float(particle[0])
    max_depth = int(np.round(particle[1]))
    max_depth = max(1, max_depth)
    n_estimators = int(np.round(particle[2]))
    n_estimators = max(1, n_estimators)
    subsample = float(particle[3])
    colsample_bytree = float(particle[4])
    gamma = float(particle[5])

    params = {
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "n_estimators": n_estimators,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "gamma": gamma,
        "random_state": RANDOM_STATE,
        "verbosity": 0,
        "objective": "reg:squarederror",
        # 'tree_method':'hist' could speed up on large data (optional)
    }
    return params

def fitness_function(particles):
    """
    particles: array shape (n_particles, dims)
    returns: array shape (n_particles,) of CV MSE (we minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)
    # We use 3-fold CV on training set and return mean MSE
    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = XGBRegressor(**params)
            # negative MSE returned -> convert to positive MSE
            cv_scores = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error", n_jobs=1)
            mse = -np.mean(cv_scores)
            # small regularization to prefer simpler models: add tiny penalty for large n_estimators / depth
            penalty = 1e-6 * (params["n_estimators"] * params["max_depth"])
            scores[i] = mse + penalty
        except Exception as e:
            # something went wrong for this particle: give a large penalty
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
# PSO settings
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}

# dimension equals number of hyperparams
dimensions = 6
# initialize optimizer
optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
    # ftol, etc. could be set; keep defaults
)

print("Starting PSO optimization (this may take some minutes depending on data size)...")
# run optimization: choose moderate iterations to balance runtime vs search
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final XGBoost with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = XGBRegressor(**best_params)
final_model.fit(X_train, y_train)

In [ ]:
y_pred_test_xg3 = final_model.predict(X_test)    # "xg" is the model and "3" is the targer which is "Purity"
y_pred_train_xg3 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_xg3.flatten()
test_vals = y_pred_test_xg3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_xg3": train_vals,
    "y_pred_test_xg3": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("xg3_predictions.xlsx", sheet_name="xg3", index=False)

print("✅ Saved to 'xg3_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_xg3, "Train")
evaluate(y_test, y_pred_test_xg3, "Test")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_xg3, alpha=0.7, edgecolor="k")
mn = min(np.min(y_test), np.min(y_pred_test_xg3))
mx = max(np.max(y_test), np.max(y_pred_test_xg3))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.title("PSO-XGBoost: Predicted vs Actual (Test)")
plt.grid(True)
plt.tight_layout()
plt.show()

## GBR-Purity

In [ ]:
y_train = df_label3.copy()
y_test = ts_label3.copy()
X_train = df.copy()
X_test = ts.copy()

In [ ]:
"""
PSO-tuned Gradient Boosting Regression (GBR)
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, max_depth, n_estimators, subsample,
         min_samples_split, min_samples_leaf, max_features
"""
from sklearn.ensemble import GradientBoostingRegressor
from pyswarms.single import GlobalBestPSO
from sklearn.model_selection import cross_val_score
import numpy as np
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Your prepared data
# ---------------------------
# Assumes these already exist exactly like in your code:
# X_train, y_train, X_test, y_test

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# We'll tune 7 parameters:
# idx 0 -> learning_rate      in [0.005, 0.30]        (float)
# idx 1 -> max_depth          in [1, 8]               (int)
# idx 2 -> n_estimators       in [50, 1000]           (int)
# idx 3 -> subsample          in [0.50, 1.00]         (float)
# idx 4 -> min_samples_split  in [2, 20]              (int)
# idx 5 -> min_samples_leaf   in [1, 10]              (int)
# idx 6 -> max_features_code  in [0.20, 1.00]         (float) -> maps to None if >=0.99, else fraction

# bounds arrays for pyswarms
lower_bounds = np.array([0.005, 1,   50, 0.50,  2,  1, 0.20])
upper_bounds = np.array([0.300, 8, 1000, 1.00, 20, 10, 1.00])

def _params_from_particle(particle):
    """Map a single particle vector to GBR params (with casts and clipping)."""
    learning_rate = float(np.clip(particle[0], lower_bounds[0], upper_bounds[0]))
    max_depth = int(np.round(np.clip(particle[1], lower_bounds[1], upper_bounds[1])))
    n_estimators = int(np.round(np.clip(particle[2], lower_bounds[2], upper_bounds[2])))
    subsample = float(np.clip(particle[3], lower_bounds[3], upper_bounds[3]))
    min_samples_split = int(np.round(np.clip(particle[4], lower_bounds[4], upper_bounds[4])))
    min_samples_leaf  = int(np.round(np.clip(particle[5], lower_bounds[5], upper_bounds[5])))

    # max_features mapping: treat near-1.0 as None (i.e., use all features)
    max_features_code = float(np.clip(particle[6], lower_bounds[6], upper_bounds[6]))
    max_features = None if max_features_code >= 0.99 else float(max_features_code)

    params = {
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "n_estimators": n_estimators,
        "subsample": subsample,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "max_features": max_features,
        "random_state": RANDOM_STATE,
    }
    return params

def fitness_function(particles):
    """
    particles: array shape (n_particles, dims)
    returns: array shape (n_particles,) of CV MSE (we minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = GradientBoostingRegressor(**params)
            # negative MSE returned -> convert to positive MSE
            cv_scores = cross_val_score(
                model, X_train, y_train,
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)

            # tiny regularization to discourage overly large trees / ensembles
            penalty = 1e-6 * (params["n_estimators"] * params["max_depth"])
            scores[i] = mse + penalty
        except Exception:
            # any failure gets a large penalty
            scores[i] = 1e10

    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 7

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),

)

print("Starting PSO optimization for GBR (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final GBR with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = GradientBoostingRegressor(**best_params)
final_model.fit(X_train, y_train)

In [ ]:
y_pred_test_gbr3 = final_model.predict(X_test)
y_pred_train_gbr3 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_gbr3.flatten()
test_vals = y_pred_test_gbr3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_gbr3": train_vals,
    "y_pred_test_gbr3": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("gbr3_predictions.xlsx", sheet_name="gbr3", index=False)

print("✅ Saved to 'gbr3_predictions.xlsx'.")

In [ ]:
y_pred_test_gbr3 = final_model.predict(X_test)
y_pred_train_gbr3 = final_model.predict(X_train)

evaluate(df_label3, y_pred_train_gbr3, "Train")
evaluate(ts_label3, y_pred_test_gbr3, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(ts_label3, y_pred_test_gbr3, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([ts_label3.min(), ts_label3.max()], [ts_label3.min(), ts_label3.max()], 'r--', lw=2)
plt.title("GBR: Predicted vs Actual (Test Set)")
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-CatBoost-Purity

In [ ]:
"""
PSO-tuned CatBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, depth, iterations, subsample, rsm, l2_leaf_reg
"""
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Data (same as your XGB block)
# ---------------------------
y_train = df_label3.copy().values  # Recovery
X_train = df.copy().values         # features (scaled)

y_test  = ts_label3.copy().values
X_test  = ts.copy().values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# dims:
# 0 -> learning_rate      [0.001, 0.3]
# 1 -> depth              [1, 10] (int)
# 2 -> iterations         [50, 1000] (int)
# 3 -> subsample          [0.3, 1.0]
# 4 -> rsm                [0.3, 1.0]  (feature subsampling)
# 5 -> l2_leaf_reg        [0.0, 10.0]

lower_bounds = np.array([0.001, 1,   50, 0.3, 0.3, 0.0])
upper_bounds = np.array([0.300, 10, 1000, 1.0, 1.0,10.0])

def _params_from_particle(p):
    """Map a particle to CatBoost params (cast + clip)."""
    learning_rate = float(np.clip(p[0], lower_bounds[0], upper_bounds[0]))
    depth         = int(np.round(np.clip(p[1], lower_bounds[1], upper_bounds[1])))
    iterations    = int(np.round(np.clip(p[2], lower_bounds[2], upper_bounds[2])))
    subsample     = float(np.clip(p[3], lower_bounds[3], upper_bounds[3]))
    rsm           = float(np.clip(p[4], lower_bounds[4], upper_bounds[4]))
    l2_leaf_reg   = float(np.clip(p[5], lower_bounds[5], upper_bounds[5]))

    params = {
        "learning_rate": learning_rate,
        "depth": depth,
        "iterations": iterations,
        "subsample": subsample,
        "rsm": rsm,
        "l2_leaf_reg": l2_leaf_reg,
        "loss_function": "RMSE",
        "random_seed": RANDOM_STATE,
        "verbose": False,
        "allow_writing_files": False,
        "bootstrap_type": "Bernoulli",   # enables 'subsample'
        "task_type": "GPU"
    }
    return params

def fitness_function(particles):
    """
    particles: (n_particles, dims)
    returns:   (n_particles,) mean CV MSE (to minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params = _params_from_particle(p)
        try:
            model = CatBoostRegressor(**params)
            # cross_val_score returns neg MSE -> take positive MSE
            cv_scores = cross_val_score(
                model, X_train, y_train,
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)
            # tiny complexity penalty (iterations * depth)
            penalty = 1e-6 * (params["iterations"] * params["depth"])
            scores[i] = mse + penalty
        except Exception:
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 6

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
)

print("Starting PSO optimization for CatBoost (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=30, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final CatBoost with best params
# ---------------------------
best_params = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_params.items():
    print(f"  {k}: {v}")

final_model = CatBoostRegressor(**best_params)
# Force CPU right before training to avoid pairwise error
final_model.set_params(task_type="CPU")   # <--- add this line
final_model.fit(X_train, y_train, verbose=False)

In [ ]:
print("CatBoostRegressor(", end="")
for i, (k, v) in enumerate(final_model.get_params().items()):
    comma = "," if i < len(final_model.get_params()) - 1 else ""
    print(f"{k}={repr(v)}{comma}")
print(")")

In [ ]:
y_pred_test_cb3 = final_model.predict(X_test)
y_pred_train_cb3 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_cb3.flatten()
test_vals = y_pred_test_cb3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_cb3": train_vals,
    "y_pred_test_cb3": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("cb3_predictions.xlsx", sheet_name="cb3", index=False)

print("✅ Saved to 'cb3_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_cb3, "Train")
evaluate(y_test, y_pred_test_cb3, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_cb3, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("CatBoost: Predicted vs Actual (Test Set)")
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.grid(True)
plt.tight_layout()
plt.show()

## PSO-AdaBoost-Purity

In [ ]:
!pip install pyswarms

In [ ]:
"""
PSO-tuned AdaBoost Regression
- Uses pyswarms GlobalBestPSO to minimize CV MSE
- Tunes: learning_rate, n_estimators, max_depth (of base tree),
         min_samples_leaf (of base tree), loss
"""
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from pyswarms.single import GlobalBestPSO
import warnings

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# ---------------------------
# 1) Data (same pattern as yours)
# ---------------------------
y_train = df_label3.copy().values  # e.g., Recovery
X_train = df.copy().values
y_test  = ts_label3.copy().values
X_test  = ts.copy().values

# ---------------------------
# 2) PSO fitness function
# ---------------------------
# dims:
# 0 -> learning_rate       [0.005, 1.0]
# 1 -> n_estimators        [50, 1000] (int)
# 2 -> max_depth           [1, 10]    (int) for base DecisionTree
# 3 -> min_samples_leaf    [1, 20]    (int) for base DecisionTree
# 4 -> loss_code           [0, 2]     (int) -> 0:'linear', 1:'square', 2:'exponential'

lower_bounds = np.array([0.005,   50,  1,  1, 0])
upper_bounds = np.array([1.000, 1000, 10, 20, 2])

_LOSSES = ['linear', 'square', 'exponential']

def _params_from_particle(p):
    """Map a particle to AdaBoost params (cast + clip)."""
    learning_rate    = float(np.clip(p[0], lower_bounds[0], upper_bounds[0]))
    n_estimators     = int(np.round(np.clip(p[1], lower_bounds[1], upper_bounds[1])))
    max_depth        = int(np.round(np.clip(p[2], lower_bounds[2], upper_bounds[2])))
    min_samples_leaf = int(np.round(np.clip(p[3], lower_bounds[3], upper_bounds[3])))
    loss_code        = int(np.round(np.clip(p[4], lower_bounds[4], upper_bounds[4])))
    loss             = _LOSSES[loss_code]

    # Base learner
    base_tree = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=RANDOM_STATE
    )

    params = {
        "estimator": base_tree,          # sklearn >= 1.2
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "loss": loss,
        "random_state": RANDOM_STATE
    }
    return params, dict(
        learning_rate=learning_rate, n_estimators=n_estimators,
        max_depth=max_depth, min_samples_leaf=min_samples_leaf, loss=loss
    )

def fitness_function(particles):
    """
    particles: (n_particles, dims)
    returns:   (n_particles,) mean CV MSE (to minimize)
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)

    for i in range(n_particles):
        p = particles[i]
        params, flat = _params_from_particle(p)
        try:
            model = AdaBoostRegressor(**params)
            # cross_val_score returns neg MSE -> convert to +MSE
            cv_scores = cross_val_score(
                model, X_train, y_train.ravel(),
                cv=3, scoring="neg_mean_squared_error", n_jobs=1
            )
            mse = -np.mean(cv_scores)
            # tiny complexity penalty to prefer simpler ensembles
            penalty = 1e-6 * (flat["n_estimators"] * flat["max_depth"])
            scores[i] = mse + penalty
        except Exception:
            scores[i] = 1e10
    return scores

# ---------------------------
# 3) Run PSO
# ---------------------------
options = {"c1": 1.5, "c2": 1.5, "w": 0.7}
dimensions = 5

optimizer = GlobalBestPSO(
    n_particles=16,
    dimensions=dimensions,
    options=options,
    bounds=(lower_bounds, upper_bounds),
)

print("Starting PSO optimization for AdaBoost (minimize 3-fold CV MSE)...")
cost, pos = optimizer.optimize(fitness_function, iters=20, verbose=True)
print("PSO finished. best cost (CV MSE):", cost)
print("Best raw particle position:", pos)

# ---------------------------
# 4) Train final AdaBoost with best params
# ---------------------------
best_params, best_flat = _params_from_particle(pos)
print("\nBest hyperparameters (rounded/casted):")
for k, v in best_flat.items():
    print(f"  {k}: {v}")

final_model = AdaBoostRegressor(**best_params)
final_model.fit(X_train, y_train.ravel())

In [ ]:
print("AdaBoostRegressor(", end="")
for i, (k, v) in enumerate(final_model.get_params().items()):
    comma = "," if i < len(final_model.get_params()) - 1 else ""
    print(f"{k}={repr(v)}{comma}")
print(")")

In [ ]:
y_pred_test_adb3 = final_model.predict(X_test)
y_pred_train_adb3 = final_model.predict(X_train)

# Make sure both arrays are 1D
train_vals = y_pred_train_adb3.flatten()
test_vals = y_pred_test_adb3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = max(len(train_vals), len(test_vals))
train_vals = np.pad(train_vals, (0, max_len - len(train_vals)), constant_values=np.nan)
test_vals = np.pad(test_vals, (0, max_len - len(test_vals)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_train_adb3": train_vals,
    "y_pred_test_adb3": test_vals
})

# Save to Excel (single sheet)
df_pred.to_excel("adb3_predictions.xlsx", sheet_name="adb3", index=False)

print("✅ Saved to 'adb3_predictions.xlsx'.")

In [ ]:
evaluate(y_train, y_pred_train_adb3, "Train")
evaluate(y_test, y_pred_test_adb3, "Test")
# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test_adb3, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("AdaBoost: Predicted vs Actual (Test Set)")
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.grid(True)
plt.tight_layout()
plt.show()

# **All Predictions based on "Total Set"**

In [ ]:
x_train = df.copy()
x_test = ts.copy()
# Recovery
y_train1 = df_label1.copy()
y_test1 = ts_label1.copy()
# SEC
y_train2 = df_label2.copy()
y_test2 = ts_label2.copy()
# Purity
y_train3 = df_label3.copy()
y_test3 = ts_label3.copy()
# Total Set
x_total = pd.concat([x_train, x_test], axis=0)
y_total1 = pd.concat([y_train1, y_test1], axis=0)
y_total2 = pd.concat([y_train2, y_test2], axis=0)
y_total3 = pd.concat([y_train3, y_test3], axis=0)

## ANN

In [ ]:
# Recovery
from tensorflow import keras
from tensorflow.keras import layers

n_features = x_total.shape[1]
# Best model
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model1.fit(x_total, y_total1, epochs=100, batch_size=32, verbose=0)
y_pred_total_ann1 = ann_model1.predict(x_total).flatten()
evaluate(y_total1, y_pred_total_ann1, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total1, y_pred_total_ann1, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total1.min(), y_total1.max()], [y_total1.min(), y_total1.max()], 'r--', lw=2)
plt.title("ANN: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# SEC
from tensorflow import keras
from tensorflow.keras import layers

n_features = x_total.shape[1]
# Best model
# ann_model2 = keras.Sequential([
#     layers.Input(shape=(n_features,)),             # 8 input features
#     layers.Dense(32, activation='relu'),   #120 128
#     layers.Dense(72, activation='relu'),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(1)                       # single output
# ])
ann_model2 = keras.Sequential([
    layers.Input(shape=(n_features,)),    # 8 input features
    layers.Dense(120, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model2.fit(x_total, y_total2, epochs=100, batch_size=32, verbose=0)
y_pred_total_ann2 = ann_model2.predict(x_total).flatten()
evaluate(y_total2, y_pred_total_ann2, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total2, y_pred_total_ann2, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total2.min(), y_total2.max()], [y_total2.min(), y_total2.max()], 'r--', lw=2)
plt.title("ANN: Predicted vs Actual (Total Set)")
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Purity
from tensorflow import keras
from tensorflow.keras import layers

# Best model
ann_model3 = keras.Sequential([
    layers.Input(shape=(n_features,)),    # 8 input features
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model3.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model3.fit(x_total, y_total3, epochs=100, batch_size=32, verbose=0)
y_pred_total_ann3 = ann_model3.predict(x_total).flatten()
evaluate(y_total3, y_pred_total_ann3, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total3, y_pred_total_ann3, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total3.min(), y_total3.max()], [y_total3.min(), y_total3.max()], 'r--', lw=2)
plt.title("ANN: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.grid(True)
plt.tight_layout()
plt.show()

**Save the 'Total Set' predictions as an excel file**

In [ ]:
total_vals1 = y_pred_total_ann1.flatten()
total_vals2 = y_pred_total_ann2.flatten()
total_vals3 = y_pred_total_ann3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = len(total_vals1)
total_vals1 = np.pad(total_vals1, (0, max_len - len(total_vals1)), constant_values=np.nan)
total_vals2 = np.pad(total_vals2, (0, max_len - len(total_vals2)), constant_values=np.nan)
total_vals3 = np.pad(total_vals3, (0, max_len - len(total_vals3)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_total_ann1": total_vals1,
    "y_pred_total_ann2": total_vals2,
    "y_pred_total_ann3": total_vals3
})

# Save to Excel (single sheet)
df_pred.to_excel("ann_total_predictions.xlsx", sheet_name="ann", index=False)

print("✅ Saved to 'ann_total_predictions.xlsx'.")

In [ ]:
y_pred_total_ann3

## TabPFN

In [ ]:
!pip install tabpfn

In [ ]:
# Recovery
from tabpfn import TabPFNRegressor

tab_model1 = TabPFNRegressor()
tab_model1.fit(x_total, y_total1)
y_pred_total_tab1 = tab_model1.predict(x_total)
evaluate(y_total1, y_pred_total_tab1, "Total")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_total1, y_pred_total_tab1, alpha=0.7, edgecolor="k")
mn = min(np.min(y_total1), np.min(y_pred_total_tab1))
mx = max(np.max(y_total1), np.max(y_pred_total_tab1))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.title("TabPFN: Predicted vs Actual (Total Set)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# SEC
from tabpfn import TabPFNRegressor

tab_model2 = TabPFNRegressor()
tab_model2.fit(x_total, y_total2)
y_pred_total_tab2 = tab_model2.predict(x_total)
evaluate(y_total2, y_pred_total_tab2, "Total")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_total2, y_pred_total_tab2, alpha=0.7, edgecolor="k")
mn = min(np.min(y_total2), np.min(y_pred_total_tab2))
mx = max(np.max(y_total2), np.max(y_pred_total_tab2))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.title("TabPFN: Predicted vs Actual (Total Set)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Purity
from tabpfn import TabPFNRegressor

tab_model3 = TabPFNRegressor()
tab_model3.fit(x_total, y_total3)
y_pred_total_tab3 = tab_model3.predict(x_total)
evaluate(y_total3, y_pred_total_tab3, "Total")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_total3, y_pred_total_tab3, alpha=0.7, edgecolor="k")
mn = min(np.min(y_total3), np.min(y_pred_total_tab3))
mx = max(np.max(y_total3), np.max(y_pred_total_tab3))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.title("TabPFN: Predicted vs Actual (Total Set)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
total_vals1 = y_pred_total_tab1.flatten()
total_vals2 = y_pred_total_tab2.flatten()
total_vals3 = y_pred_total_tab3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = len(total_vals1)
total_vals1 = np.pad(total_vals1, (0, max_len - len(total_vals1)), constant_values=np.nan)
total_vals2 = np.pad(total_vals2, (0, max_len - len(total_vals2)), constant_values=np.nan)
total_vals3 = np.pad(total_vals3, (0, max_len - len(total_vals3)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_total_tab1": total_vals1,
    "y_pred_total_tab2": total_vals2,
    "y_pred_total_tab3": total_vals3
})

# Save to Excel (single sheet)
df_pred.to_excel("tab_total_predictions.xlsx", sheet_name="tab", index=False)

print("✅ Saved to 'tab_total_predictions.xlsx'.")

## PSO-XGBoost

In [ ]:
# Recovery
from xgboost import XGBRegressor
xgb_model1 = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.794347904457994, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=0.45398325921376137, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.08564167838681164,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, monotone_constraints=None,
             multi_strategy=None, n_estimators=724, n_jobs=None)

xgb_model1.fit(x_total, y_total1)
y_pred_total_xg1 = xgb_model1.predict(x_total)
evaluate(y_total1, y_pred_total_xg1, "Total")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_total1, y_pred_total_xg1, alpha=0.7, edgecolor="k")
mn = min(np.min(y_total1), np.min(y_pred_total_xg1))
mx = max(np.max(y_total1), np.max(y_pred_total_xg1))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.title("PSO-XGBoost: Predicted vs Actual (Total Set)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# SEC
from xgboost import XGBRegressor
xgb_model2 = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9207502204636566, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=4.562640895437547, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.033250273864687305,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, monotone_constraints=None,
             multi_strategy=None, n_estimators=672, n_jobs=None)

xgb_model2.fit(x_total, y_total2)
y_pred_total_xg2 = xgb_model2.predict(x_total)
evaluate(y_total2, y_pred_total_xg2, "Total")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_total2, y_pred_total_xg2, alpha=0.7, edgecolor="k")
mn = min(np.min(y_total2), np.min(y_pred_total_xg2))
mx = max(np.max(y_total2), np.max(y_pred_total_xg2))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.title("PSO-XGBoost: Predicted vs Actual (Total Set)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Purity
from xgboost import XGBRegressor
xgb_model3 = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9727111775192585, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=0.5320395779360891, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.041150584937100235,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, monotone_constraints=None,
             multi_strategy=None, n_estimators=938, n_jobs=None)

xgb_model3.fit(x_total, y_total3)
y_pred_total_xg3 = xgb_model3.predict(x_total)
evaluate(y_total3, y_pred_total_xg3, "Total")

# Plot Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_total3, y_pred_total_xg3, alpha=0.7, edgecolor="k")
mn = min(np.min(y_total3), np.min(y_pred_total_xg3))
mx = max(np.max(y_total3), np.max(y_pred_total_xg3))
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2)
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.title("PSO-XGBoost: Predicted vs Actual (Total Set)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
total_vals1 = y_pred_total_xg1.flatten()
total_vals2 = y_pred_total_xg2.flatten()
total_vals3 = y_pred_total_xg3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = len(total_vals1)
total_vals1 = np.pad(total_vals1, (0, max_len - len(total_vals1)), constant_values=np.nan)
total_vals2 = np.pad(total_vals2, (0, max_len - len(total_vals2)), constant_values=np.nan)
total_vals3 = np.pad(total_vals3, (0, max_len - len(total_vals3)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_total_xg1": total_vals1,
    "y_pred_total_xg2": total_vals2,
    "y_pred_total_xg3": total_vals3
})

# Save to Excel (single sheet)
df_pred.to_excel("xg_total_predictions.xlsx", sheet_name="xg", index=False)

print("✅ Saved to 'xg_total_predictions.xlsx'.")

## PSO-GBR

In [ ]:
# Recovery
from sklearn.ensemble import GradientBoostingRegressor
gbr_model1 = GradientBoostingRegressor(learning_rate=0.07780212667426754, max_depth=4,
                          max_features=0.7150294538727542, min_samples_leaf=4,
                          min_samples_split=14, n_estimators=633,
                          random_state=42, subsample=0.7757643293874783)

gbr_model1.fit(x_total, y_total1)
y_pred_total_gbr1 = gbr_model1.predict(x_total)
evaluate(y_total1, y_pred_total_gbr1, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total1, y_pred_total_gbr1, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total1.min(), y_total1.max()], [y_total1.min(), y_total1.max()], 'r--', lw=2)
plt.title("GBR: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# SEC
from sklearn.ensemble import GradientBoostingRegressor
gbr_model2 = GradientBoostingRegressor(learning_rate=0.14250485305726465, max_depth=4,
                          max_features=0.9075248053662535, min_samples_leaf=2,
                          min_samples_split=18, n_estimators=607,
                          random_state=42, subsample=0.9951297938254593)

gbr_model2.fit(x_total, y_total2)
y_pred_total_gbr2 = gbr_model2.predict(x_total)
evaluate(y_total2, y_pred_total_gbr2, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total2, y_pred_total_gbr2, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total2.min(), y_total2.max()], [y_total2.min(), y_total2.max()], 'r--', lw=2)
plt.title("GBR: Predicted vs Actual (Total Set)")
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Purity
from sklearn.ensemble import GradientBoostingRegressor
gbr_model3 = GradientBoostingRegressor(learning_rate=0.028832399541631026, max_depth=8,
                          max_features=0.9306761987596761, min_samples_leaf=7,
                          min_samples_split=11, n_estimators=919,
                          random_state=42, subsample=0.5396359220219887)

gbr_model3.fit(x_total, y_total3)
y_pred_total_gbr3 = gbr_model3.predict(x_total)
evaluate(y_total3, y_pred_total_gbr3, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total3, y_pred_total_gbr3, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total3.min(), y_total3.max()], [y_total3.min(), y_total3.max()], 'r--', lw=2)
plt.title("GBR: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
total_vals1 = y_pred_total_gbr1.flatten()
total_vals2 = y_pred_total_gbr2.flatten()
total_vals3 = y_pred_total_gbr3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = len(total_vals1)
total_vals1 = np.pad(total_vals1, (0, max_len - len(total_vals1)), constant_values=np.nan)
total_vals2 = np.pad(total_vals2, (0, max_len - len(total_vals2)), constant_values=np.nan)
total_vals3 = np.pad(total_vals3, (0, max_len - len(total_vals3)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_total_gbr1": total_vals1,
    "y_pred_total_gbr2": total_vals2,
    "y_pred_total_gbr3": total_vals3
})

# Save to Excel (single sheet)
df_pred.to_excel("gbr_total_predictions.xlsx", sheet_name="gbr", index=False)

print("✅ Saved to 'gbr_total_predictions.xlsx'.")

## PSO-CatBoost

In [ ]:
!pip install catboost

In [ ]:
# Recovery
from catboost import CatBoostRegressor
cb_model1 = CatBoostRegressor(iterations=987, learning_rate=0.06921084015993914,
                              depth=7, l2_leaf_reg=1.6926258102111902, rsm=0.8583651828877268,
                              loss_function='RMSE', random_seed=42, verbose=False,
                              allow_writing_files=False, task_type='CPU',
                              bootstrap_type='Bernoulli', subsample=0.8125328711078112)

cb_model1.fit(x_total, y_total1)
y_pred_total_cb1 = cb_model1.predict(x_total)
evaluate(y_total1, y_pred_total_cb1, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total1, y_pred_total_cb1, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total1.min(), y_total1.max()], [y_total1.min(), y_total1.max()], 'r--', lw=2)
plt.title("CatBoost: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# SEC
from catboost import CatBoostRegressor
cb_model2 = CatBoostRegressor(iterations=484, learning_rate=0.04463768687603424, depth=5,
                              l2_leaf_reg=0.5987692889341945, rsm=0.6484702678951451,
                              loss_function='RMSE', random_seed=42, verbose=False,
                              allow_writing_files=False, task_type='CPU',
                              bootstrap_type='Bernoulli', subsample=0.8601134735851019)

cb_model2.fit(x_total, y_total2)
y_pred_total_cb2 = cb_model2.predict(x_total)
evaluate(y_total2, y_pred_total_cb2, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total2, y_pred_total_cb2, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total2.min(), y_total2.max()], [y_total2.min(), y_total2.max()], 'r--', lw=2)
plt.title("CatBoost: Predicted vs Actual (Total Set)")
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Purity
from catboost import CatBoostRegressor
cb_model3 = CatBoostRegressor(iterations=54, learning_rate=0.13410536212915647, depth=8,
                              l2_leaf_reg=0.376584630037079, rsm=0.9710562268314031,
                              loss_function='RMSE', random_seed=42, verbose=False,
                              allow_writing_files=False, task_type='CPU',
                              bootstrap_type='Bernoulli', subsample=0.5895788236923274)

cb_model3.fit(x_total, y_total3)
y_pred_total_cb3 = cb_model3.predict(x_total)
evaluate(y_total3, y_pred_total_cb3, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total3, y_pred_total_cb3, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total3.min(), y_total3.max()], [y_total3.min(), y_total3.max()], 'r--', lw=2)
plt.title("CatBoost: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
total_vals1 = y_pred_total_cb1.flatten()
total_vals2 = y_pred_total_cb2.flatten()
total_vals3 = y_pred_total_cb3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = len(total_vals1)
total_vals1 = np.pad(total_vals1, (0, max_len - len(total_vals1)), constant_values=np.nan)
total_vals2 = np.pad(total_vals2, (0, max_len - len(total_vals2)), constant_values=np.nan)
total_vals3 = np.pad(total_vals3, (0, max_len - len(total_vals3)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_total_cb1": total_vals1,
    "y_pred_total_cb2": total_vals2,
    "y_pred_total_cb3": total_vals3
})

# Save to Excel (single sheet)
df_pred.to_excel("cb_total_predictions.xlsx", sheet_name="cb", index=False)

print("✅ Saved to 'cb_total_predictions.xlsx'.")

## PSO-AdaBoost

In [ ]:
# Recovery
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Define base estimator
base_tree = DecisionTreeRegressor(
    criterion='squared_error',
    max_depth=10,
    min_samples_leaf=3,
    random_state=42
)

# Define AdaBoost model
adb_model1 = AdaBoostRegressor(
    estimator=base_tree,
    learning_rate=0.9007685647106364,
    loss='square',
    n_estimators=471,
    random_state=42
)

# Fit model
adb_model1.fit(x_total, y_total1)

# Predict
y_pred_total_adb1 = adb_model1.predict(x_total)
evaluate(y_total1, y_pred_total_adb1, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total1, y_pred_total_adb1, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total1.min(), y_total1.max()], [y_total1.min(), y_total1.max()], 'r--', lw=2)
plt.title("AdaBoost: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Recovery%")
plt.ylabel("Predicted Recovery%")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# SEC
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Define base estimator
base_tree = DecisionTreeRegressor(
    criterion='squared_error',
    max_depth=10,
    min_samples_leaf=3,
    random_state=42
)

# Define AdaBoost model
adb_model2 = AdaBoostRegressor(
    estimator=base_tree,
    learning_rate=0.479481012015941,
    loss='square',
    n_estimators=593,
    random_state=42
)

# Fit model
adb_model2.fit(x_total, y_total2)

# Predict
y_pred_total_adb2 = adb_model2.predict(x_total)
evaluate(y_total2, y_pred_total_adb2, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total2, y_pred_total_adb2, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total2.min(), y_total2.max()], [y_total2.min(), y_total2.max()], 'r--', lw=2)
plt.title("AdaBoost: Predicted vs Actual (Total Set)")
plt.xlabel("Actual SEC")
plt.ylabel("Predicted SEC")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Purity
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Define base estimator
base_tree = DecisionTreeRegressor(
    criterion='squared_error',
    max_depth=10,
    min_samples_leaf=10,
    random_state=42
)

# Define AdaBoost model
adb_model3 = AdaBoostRegressor(
    estimator=base_tree,
    learning_rate=0.03597743718960905,
    loss='square',
    n_estimators=874,
    random_state=42
)

# Fit model
adb_model3.fit(x_total, y_total3)

# Predict
y_pred_total_adb3 = adb_model3.predict(x_total)
evaluate(y_total3, y_pred_total_adb3, "Total")

# Visualization
plt.figure(figsize=(6,6))
plt.scatter(y_total3, y_pred_total_adb3, color='dodgerblue', alpha=0.7, edgecolor='k')
plt.plot([y_total3.min(), y_total3.max()], [y_total3.min(), y_total3.max()], 'r--', lw=2)
plt.title("AdaBoost: Predicted vs Actual (Total Set)")
plt.xlabel("Actual Purity")
plt.ylabel("Predicted Purity")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
total_vals1 = y_pred_total_adb1.flatten()
total_vals2 = y_pred_total_adb2.flatten()
total_vals3 = y_pred_total_adb3.flatten()

# Pad shorter array with NaN so they align in Excel
max_len = len(total_vals1)
total_vals1 = np.pad(total_vals1, (0, max_len - len(total_vals1)), constant_values=np.nan)
total_vals2 = np.pad(total_vals2, (0, max_len - len(total_vals2)), constant_values=np.nan)
total_vals3 = np.pad(total_vals3, (0, max_len - len(total_vals3)), constant_values=np.nan)

# Create the DataFrame with exactly two columns
df_pred = pd.DataFrame({
    "y_pred_total_adb1": total_vals1,
    "y_pred_total_adb2": total_vals2,
    "y_pred_total_adb3": total_vals3
})

# Save to Excel (single sheet)
df_pred.to_excel("adb_total_predictions.xlsx", sheet_name="adb", index=False)

print("✅ Saved to 'adb_total_predictions.xlsx'.")

# **Results and Comparison**

## Plots and Metrics

In [ ]:
from google.colab import files

uploaded = files.upload()
file_name2 = list(uploaded.keys())[0]
df_pred = pd.read_excel(file_name2)
all_sheets = pd.read_excel("predictions.xlsx", sheet_name=None)

## --------------------------------------------

# df_pred = pd.read_excel("C:\\Users\\Pgshco\\Desktop\\Python ML\\Carbon Capture\\ML codes\\Final Code\\predictions.xlsx")
# all_sheets = pd.read_excel("C:\\Users\\Pgshco\\Desktop\\Python ML\\Carbon Capture\\ML codes\\Final Code\\predictions.xlsx", sheet_name=None)

In [ ]:
import pandas as pd


# Loop through each sheet and create variables dynamically
for sheet_name, df_pred in all_sheets.items():
    model_name = ''.join([c for c in sheet_name if not c.isdigit()])  # e.g., 'gbr'
    target_num = ''.join([c for c in sheet_name if c.isdigit()])      # e.g., '3'

    # Extract columns
    train_col = df_pred.iloc[:, 0].values
    test_col  = df_pred.iloc[:, 1].values

    # Create variable names like y_pred_train_gbr3, y_pred_test_gbr3
    globals()[f"y_pred_train_{model_name}{target_num}"] = train_col
    globals()[f"y_pred_test_{model_name}{target_num}"]  = test_col

    print(f"Loaded: y_pred_train_{model_name}{target_num}, y_pred_test_{model_name}{target_num}")


In [ ]:
import numpy as np
### Train
# Iterate over a snapshot of current globals to avoid runtime error
for name in list(globals().keys()):
    value = globals()[name]
    if name.startswith("y_pred_train_") and isinstance(value, np.ndarray):
        cleaned = value[~np.isnan(value)]   # remove NaN values
        globals()[name] = cleaned
        print(f"{name}: cleaned -> {len(cleaned)} values")



### Test
# Iterate over a snapshot of current globals to avoid runtime error
for name in list(globals().keys()):
    value = globals()[name]
    if name.startswith("y_pred_test_") and isinstance(value, np.ndarray):
        cleaned = value[~np.isnan(value)]   # remove NaN values
        globals()[name] = cleaned
        print(f"{name}: cleaned -> {len(cleaned)} values")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ===== Prepare true and predicted values =====
y_true_train_rec = np.asarray(df_label1).ravel()
y_true_train_sec = np.asarray(df_label2).ravel()
y_true_train_pur = np.asarray(df_label3).ravel()

y_true_test_rec = np.asarray(ts_label1).ravel()
y_true_test_sec = np.asarray(ts_label2).ravel()
y_true_test_pur = np.asarray(ts_label3).ravel()

y_pred_train_rec = y_pred_train_ann1
y_pred_train_sec = y_pred_train_ann2
y_pred_train_pur = y_pred_train_ann3

y_pred_test_rec = y_pred_test_ann1
y_pred_test_sec = y_pred_test_ann2
y_pred_test_pur = y_pred_test_ann3

eps = 1e-8


####### Recovery ##########
plt.figure(figsize=(8, 6))

# Train
rel_train = (y_pred_train_rec - y_true_train_rec) / (y_true_train_rec + eps) * 100
plt.scatter(y_true_train_rec, rel_train,
            s=45, alpha=0.75, color='red',
            edgecolor='black', linewidth=0.4, label='Train')

# Test
rel_test = (y_pred_test_rec - y_true_test_rec) / (y_true_test_rec + eps) * 100
plt.scatter(y_true_test_rec, rel_test,
            s=45, alpha=0.75, color='blue',
            edgecolor='black', linewidth=0.4, label='Test')

plt.axhline(0, color='black', linestyle='--')
plt.xlabel("Actual Recovery (%)", fontsize=14)
plt.ylabel("Relative deviation (%)", fontsize=14)
plt.title("ANN – Relative deviation for Recovery (%)", fontsize=16)
plt.legend(fontsize=14)
plt.grid(True, linestyle='--', alpha=0.45)

plt.tight_layout()
plt.savefig("Figures/rel_dev_recovery.png", dpi=600, bbox_inches="tight")
plt.show()

####### SEC ##########
plt.figure(figsize=(8, 6))

rel_train = (y_pred_train_sec - y_true_train_sec) / (y_true_train_sec + eps) * 100
plt.scatter(y_true_train_sec, rel_train,
            s=45, alpha=0.75, color='red',
            edgecolor='black', linewidth=0.4, label='Train')

rel_test = (y_pred_test_sec - y_true_test_sec) / (y_true_test_sec + eps) * 100
plt.scatter(y_true_test_sec, rel_test,
            s=45, alpha=0.75, color='blue',
            edgecolor='black', linewidth=0.4, label='Test')

plt.axhline(0, color='black', linestyle='--')
plt.xlabel("Actual SEC (kJ/kg)", fontsize=14)
plt.ylabel("Relative deviation (%)", fontsize=14)
plt.title("ANN – Relative deviation for SEC (kJ/kg)", fontsize=16)
plt.legend(fontsize=14)
plt.grid(True, linestyle='--', alpha=0.45)

plt.tight_layout()
plt.savefig("Figures/rel_dev_sec.png", dpi=600, bbox_inches="tight")
plt.show()

####### Purity ###########
plt.figure(figsize=(8, 6))

rel_train = (y_pred_train_pur - y_true_train_pur) / (y_true_train_pur + eps) * 100
plt.scatter(y_true_train_pur, rel_train,
            s=45, alpha=0.75, color='red',
            edgecolor='black', linewidth=0.4, label='Train')

rel_test = (y_pred_test_pur - y_true_test_pur) / (y_true_test_pur + eps) * 100
plt.scatter(y_true_test_pur, rel_test,
            s=45, alpha=0.75, color='blue',
            edgecolor='black', linewidth=0.4, label='Test')

plt.axhline(0, color='black', linestyle='--')
plt.xlabel("Actual Purity (mol%)", fontsize=14)
plt.ylabel("Relative deviation (%)", fontsize=14)
plt.title("ANN – Relative deviation for Purity (mol%)", fontsize=16)
plt.legend(fontsize=14)
plt.grid(True, linestyle='--', alpha=0.45)

plt.tight_layout()
plt.savefig("Figures/rel_dev_purity.png", dpi=600, bbox_inches="tight")
plt.show()


In [ ]:
import pandas as pd

# --- True values mapping ---
true_values = {
    1: (df_label1, ts_label1),
    2: (df_label2, ts_label2),
    3: (df_label3, ts_label3)
}

models = ["ann", "tab", "xg", "gbr", "cb", "adb"]
metric_names = ["R²", "Adj-R²", "MSE", "RMSE", "NRMSE", "std_er","MAE", "MAPE", "MedAE", "EVS"]

results = []

# --- Loop through each model and target ---
for model in models:
    for target in [1, 2, 3]:
        y_train_true, y_test_true = true_values[target]
        y_pred_train_var = f"y_pred_train_{model}{target}"
        y_pred_test_var  = f"y_pred_test_{model}{target}"

        # Skip missing variables
        if y_pred_test_var not in globals():
            print(f"⚠️ Skipping {y_pred_test_var} (not found)")
            continue

        y_pred_train = globals()[y_pred_train_var]
        y_pred_test  = globals()[y_pred_test_var]

        # Evaluate
        train_metrics = evaluate(y_train_true, y_pred_train, label="Train")
        test_metrics  = evaluate(y_test_true,  y_pred_test,  label="Test")

        # Combine metrics side-by-side (Train_Test pairs)
        combined_metrics = {"Model": model.upper(), "Target": {1:"Recovery",2:"SEC",3:"Purity"}[target]}
        for i, metric in enumerate(metric_names):
            combined_metrics[f"{metric}_Train"] = train_metrics[i]
            combined_metrics[f"{metric}_Test"]  = test_metrics[i]

        results.append(combined_metrics)

# --- Create final DataFrame ---
metrics_table = pd.DataFrame(results)

# --- Display neatly ---
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
display(metrics_table)

# --- Save to Excel ---
metrics_table.to_excel("All_Model_Metrics.xlsx", index=False)
print("✅ Saved metrics table with Train/Test pairs to 'All_Model_Metrics.xlsx'")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Explicit mapping for true values (no globals lookups)
true_values = {
    1: np.asarray(ts_label1, dtype=float),
    2: np.asarray(ts_label2, dtype=float),
    3: np.asarray(ts_label3, dtype=float),
}

models = [
    ("ann", "ANN"),
    ("tab", "TabPFN"),
    ("xg",  "PSO-XGBoost"),
    ("gbr", "PSO-GBR"),
    ("cb", "PSO-CatBoost"),
    ("adb", "PSO-AdaBoost")
]
target_names = {1: "Recovery", 2: "SEC", 3: "Purity"}

fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(14, 20))
for r, (mkey, mname) in enumerate(models):
    for c, t in enumerate([1, 2, 3]):
        ax = axes[r, c]
        y_true = true_values[t]
        # predictions still fetched by name:
        y_pred = np.asarray(globals()[f"y_pred_test_{mkey}{t}"], dtype=float)

        mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
        yt, yp = y_true[mask], y_pred[mask]

        ax.scatter(yt, yp, alpha=0.7, edgecolor="k")
        mn, mx = float(min(yt.min(), yp.min())), float(max(yt.max(), yp.max()))
        ax.plot([mn, mx], [mn, mx], "r--", linewidth=2)

        tname = target_names[t]
        ax.set_xlabel(f"Actual {tname}")
        ax.set_ylabel(f"Predicted {tname}")
        ax.set_title(f"{mname}: Predicted vs Actual (Test)")
        ax.grid(True)

        # Format SEC axes as ×10^5
        if t == 2:
            ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x/1e5:.0f}"))
            ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y/1e5:.0f}"))
            ax.text(0.98, -0.14, r"$\times10^{5}$",
                    transform=ax.transAxes, ha="right", va="center", fontsize=10)

plt.tight_layout()
plt.savefig("All_Model_Scatter.svg", format="svg", dpi=300)
plt.show()
print("✅ Saved as All_Model_Scatter.svg")


## 5-fold cross-validation

### Recovery

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Assuming your DataFrame is df
# X = features, y = target

X = x_total.values  # all columns except last
y = y_total1.values # Recovery

# Define a function to create the ANN model
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=input_dim))
    model.add(Dense(72, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))  # regression output
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# K-Fold Cross Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

r2_scores = []
rmse_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_model(input_dim=X.shape[1])

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Predict on validation set
    y_pred = model.predict(X_val).flatten()

    # Evaluate metrics
    r2 = r2_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    r2_scores.append(r2)
    rmse_scores.append(rmse)

    print(f"Fold {fold}: R2 = {r2:.4f}, RMSE = {rmse:.4f}")

# Final average metrics
print(f"\nAverage R2: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")

### SEC

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Assuming your DataFrame is df
# X = features, y = target

X = x_total.values  # all columns except last
y = y_total2.values # Recovery

# Define a function to create the ANN model
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(120, activation='relu', input_dim=input_dim))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # regression output
    model.compile(optimizer=Adam(learning_rate=0.1), loss='mse')
    return model

# K-Fold Cross Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

r2_scores = []
rmse_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_model(input_dim=X.shape[1])

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Predict on validation set
    y_pred = model.predict(X_val).flatten()

    # Evaluate metrics
    r2 = r2_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    r2_scores.append(r2)
    rmse_scores.append(rmse)

    print(f"Fold {fold}: R2 = {r2:.4f}, RMSE = {rmse:.4f}")

# Final average metrics
print(f"\nAverage R2: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")


SEC with the tunned hyper parameter of Rec and Pu

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Assuming your DataFrame is df
# X = features, y = target

X = x_total.values  # all columns except last
y = y_total2.values # Recovery

# Define a function to create the ANN model
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=input_dim))
    model.add(Dense(72, activation='relu', input_dim=input_dim))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))  # regression output
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# K-Fold Cross Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

r2_scores = []
rmse_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_model(input_dim=X.shape[1])

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Predict on validation set
    y_pred = model.predict(X_val).flatten()

    # Evaluate metrics
    r2 = r2_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    r2_scores.append(r2)
    rmse_scores.append(rmse)

    print(f"Fold {fold}: R2 = {r2:.4f}, RMSE = {rmse:.4f}")

# Final average metrics
print(f"\nAverage R2: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")


### Purity

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Assuming your DataFrame is df
# X = features, y = target

X = x_total.values  # all columns except last
y = y_total3.values # Recovery

# Define a function to create the ANN model
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=input_dim))
    model.add(Dense(72, activation='relu', input_dim=input_dim))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))  # regression output
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# K-Fold Cross Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

r2_scores = []
rmse_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_model(input_dim=X.shape[1])

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Predict on validation set
    y_pred = model.predict(X_val).flatten()

    # Evaluate metrics
    r2 = r2_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    r2_scores.append(r2)
    rmse_scores.append(rmse)

    print(f"Fold {fold}: R2 = {r2:.4f}, RMSE = {rmse:.4f}")

# Final average metrics
print(f"\nAverage R2: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")

## Shap

### Recovery

In [ ]:
!pip install shap

In [ ]:
!pip install keras

In [ ]:
import shap
from shap import summary_plot

# Model
# Recovery
from tensorflow import keras
from tensorflow.keras import layers

X_train = df
y_train1 = df_label1
X_test = ts
n_features = X_train.shape[1]
# Best model
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model1.fit(X_train, y_train1, epochs=100, batch_size=32, verbose=0)


feature_names = list(df.columns[:-1])
explainer = shap.Explainer(ann_model1.predict, X_train, feature_names = feature_names)
shap_values = explainer(X_test)

In [ ]:
import matplotlib.pyplot as plt
import shap
import numpy as np

# Column index for "Unnamed: 0"
col_index = X_test.columns.get_loc("Unnamed: 0")

# 1. Drop column from the feature dataframe
X_test_clean = X_test.drop(columns=["Unnamed: 0"])

# 2. Remove the same column from SHAP values
values_clean = np.delete(shap_values.values, col_index, axis=1)

# Optional: SHAP stores the original features in .data
if shap_values.data is not None:
    data_clean = np.delete(shap_values.data, col_index, axis=1)
else:
    data_clean = None

# Rebuild the Explanation object
shap_values_clean = shap.Explanation(
    values=values_clean,
    base_values=shap_values.base_values,
    data=data_clean,
    feature_names=[name for i, name in enumerate(shap_values.feature_names) if i != col_index]
)
# --- BAR PLOT ---
plt.figure(figsize=(20, 16))   # <<< CRITICAL FIX
shap.summary_plot(
    shap_values_clean,
    X_test_clean,
    plot_type="bar",
    show=False                # prevent SHAP from clearing the figure
)
plt.gca().set_xlabel("mean(|SHAP value|)")

plt.tight_layout()
plt.show()


# --- DOT PLOT ---
shap.summary_plot(shap_values_clean, X_test_clean)
plt.tight_layout()


In [ ]:
import matplotlib.pyplot as plt
import shap

# --- BAR PLOT ---
shap.summary_plot(shap_values_clean, X_test_clean, plot_type="bar", show=False)
plt.gca().set_xlabel("mean(|SHAP value|)")
plt.tight_layout()
plt.savefig("shap_rec_bar.png", dpi=600, bbox_inches='tight')
plt.close()

# --- DOT PLOT ---
shap.summary_plot(shap_values_clean, X_test_clean, show=False)
plt.tight_layout()
plt.savefig("shap_rec_dot.png", dpi=600, bbox_inches='tight')
plt.close()


### SEC

In [ ]:
!pip install shap
!pip install keras

In [ ]:
import shap
from shap import summary_plot

# Model
# Recovery
from tensorflow import keras
from tensorflow.keras import layers

X_train = df.drop(columns=['Unnamed: 0'])
y_train2 = df_label2
X_test = ts.drop(columns=['Unnamed: 0'])
n_features = X_train.shape[1]
# Best model
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(120, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model1.fit(X_train, y_train2, epochs=101, batch_size=32, verbose=0)


feature_names = list(df.columns[:-1])
explainer = shap.Explainer(ann_model1.predict, X_train, feature_names = feature_names)
shap_values = explainer(X_test)

In [ ]:
# --- BAR PLOT ---
shap.summary_plot(
    shap_values,
    X_test,
    plot_type="bar",
    show=False                # prevent SHAP from clearing the figure
)
plt.gca().set_xlabel("mean(|SHAP value|)")

plt.tight_layout()
plt.show()


# --- DOT PLOT ---
shap.summary_plot(shap_values, X_test)
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import shap

# --- BAR PLOT ---
shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
plt.gca().set_xlabel("mean(|SHAP value|)")
plt.tight_layout()
plt.savefig("shap_sec_bar.png", dpi=600, bbox_inches='tight')
plt.close()

# --- DOT PLOT ---
shap.summary_plot(shap_values, X_test, show=False)
plt.tight_layout()
plt.savefig("shap_sec_dot.png", dpi=600, bbox_inches='tight')
plt.close()

### Purity

In [ ]:
import shap
from shap import summary_plot

# Model
# Recovery
from tensorflow import keras
from tensorflow.keras import layers

X_train = df.drop(columns=['Unnamed: 0'])
y_train3 = df_label3
X_test = ts.drop(columns=['Unnamed: 0'])
n_features = X_train.shape[1]
# Best model
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model1.fit(X_train, y_train3, epochs=100, batch_size=32, verbose=0)


feature_names = list(df.columns[:-1])
explainer = shap.Explainer(ann_model1.predict, X_train, feature_names = feature_names)
shap_values = explainer(X_test)

In [ ]:
# --- BAR PLOT ---
shap.summary_plot(
    shap_values,
    X_test,
    plot_type="bar",
    show=False                # prevent SHAP from clearing the figure
)
plt.gca().set_xlabel("mean(|SHAP value|)")

plt.tight_layout()
plt.show()


# --- DOT PLOT ---
shap.summary_plot(shap_values, X_test)
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import shap

# --- BAR PLOT ---
shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
plt.gca().set_xlabel("mean(|SHAP value|)")
plt.tight_layout()
plt.savefig("shap_pur_bar.png", dpi=600, bbox_inches='tight')
plt.close()

# --- DOT PLOT ---
shap.summary_plot(shap_values, X_test, show=False)
plt.tight_layout()
plt.savefig("shap_pur_dot.png", dpi=600, bbox_inches='tight')
plt.close()

## William's Plot

### **Recovery**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers


# محاسبه استاندارد شده‌ی باقیمانده‌ها و leverage
y_true = df_label1
X_train = df.drop(columns=['Unnamed: 0'])


# model
n_features = X_train.shape[1]
# Best model
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model1.fit(X_train, y_true, epochs=101, batch_size=32, verbose=0)


y_pred = ann_model1.predict(X_train)
y_pred = np.ravel(y_pred)          # (3540,)
y_true = np.ravel(y_true)          # (3540,)
residuals = y_true - y_pred
std_residuals = residuals / np.std(residuals)

# طراحی X با ستون ثابت
X_design = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
XT_X_inv = np.linalg.inv(X_design.T.dot(X_design))
H = X_design @ XT_X_inv @ X_design.T
leverage = np.diag(H)

# آستانه‌ها
n, p = X_train.shape
h_star = 3*(p+1)/n
resid_thresh = 3

# دسته‌بندی نقاط
valid_idx     = (np.abs(std_residuals) <= resid_thresh) & (leverage <= h_star)
suspect_idx   = (np.abs(std_residuals) >  resid_thresh) & (leverage <= h_star)
leverage_idx  = (leverage > h_star) & (np.abs(std_residuals) <= resid_thresh)
both_idx      = (leverage > h_star) & (np.abs(std_residuals) > resid_thresh)

# رسم نمودار
plt.figure(figsize=(7, 4))
plt.axhline(resid_thresh,  color='darkorange', linestyle='--', label='Upper suspected limit')
plt.axhline(-resid_thresh, color='darkorange', linestyle='--', label='Lower suspected limit')
plt.axvline(h_star,        color='purple', linestyle='--', label='Out of Leverage')

# نقاط
plt.scatter(leverage[valid_idx], std_residuals[valid_idx],   c='darkblue', label='Valid data', s=50)
plt.scatter(leverage[suspect_idx], std_residuals[suspect_idx], c='red', label='Suspected data', edgecolor='black', s=80)
plt.scatter(leverage[leverage_idx], std_residuals[leverage_idx], c='lime', label='Out of Leverage', edgecolor='black', s=80)
plt.scatter(leverage[both_idx], std_residuals[both_idx], c='maroon', label='Suspected & Out of Leverage', s=90, edgecolor='black')

plt.xlabel('Hat')
plt.ylabel('Standardized residuals')
plt.title('Leverage vs Standardized Residuals (ANN-Recovery)')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("leverage_residuals_rec.png", dpi=600, bbox_inches='tight')
plt.show()

# Calculate the number of points in each category
num_valid = np.sum(valid_idx)
num_suspect = np.sum(suspect_idx)
num_out_of_leverage = np.sum(leverage_idx)
num_both = np.sum(both_idx)

# Total points
total_points = len(X_train)

# Calculate the percentages
valid_percentage = (num_valid / total_points) * 100
suspect_percentage = (num_suspect / total_points) * 100
out_of_leverage_percentage = (num_out_of_leverage / total_points) * 100
both_percentage = (num_both / total_points) * 100

# Print results
print(f"Number of valid data points: {num_valid} ({valid_percentage:.2f}%)")
print(f"Number of suspected data points: {num_suspect} ({suspect_percentage:.2f}%)")
print(f"Number of out of leverage data points: {num_out_of_leverage} ({out_of_leverage_percentage:.2f}%)")
print(f"Number of suspected & out of leverage data points: {num_both} ({both_percentage:.2f}%)")

### **SEC**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers


# محاسبه استاندارد شده‌ی باقیمانده‌ها و leverage
y_true = df_label2
X_train = df.drop(columns=['Unnamed: 0'])

# model
n_features = X_train.shape[1]
# Best model
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(120, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model1.fit(X_train, y_true, epochs=101, batch_size=32, verbose=0)

y_pred = ann_model1.predict(X_train)
y_pred = np.ravel(y_pred)          # (3540,)
y_true = np.ravel(y_true)          # (3540,)
residuals = y_true - y_pred
std_residuals = residuals / np.std(residuals)

# طراحی X با ستون ثابت
X_design = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
XT_X_inv = np.linalg.inv(X_design.T.dot(X_design))
H = X_design @ XT_X_inv @ X_design.T
leverage = np.diag(H)

# آستانه‌ها
n, p = X_train.shape
h_star = 3*(p+1)/n
resid_thresh = 3

# دسته‌بندی نقاط
valid_idx     = (np.abs(std_residuals) <= resid_thresh) & (leverage <= h_star)
suspect_idx   = (np.abs(std_residuals) >  resid_thresh) & (leverage <= h_star)
leverage_idx  = (leverage > h_star) & (np.abs(std_residuals) <= resid_thresh)
both_idx      = (leverage > h_star) & (np.abs(std_residuals) > resid_thresh)

# رسم نمودار
plt.figure(figsize=(7, 4))
plt.axhline(resid_thresh,  color='darkorange', linestyle='--', label='Upper suspected limit')
plt.axhline(-resid_thresh, color='darkorange', linestyle='--', label='Lower suspected limit')
plt.axvline(h_star,        color='purple', linestyle='--', label='Out of Leverage')

# نقاط
plt.scatter(leverage[valid_idx], std_residuals[valid_idx],   c='darkblue', label='Valid data', s=50)
plt.scatter(leverage[suspect_idx], std_residuals[suspect_idx], c='red', label='Suspected data', edgecolor='black', s=80)
plt.scatter(leverage[leverage_idx], std_residuals[leverage_idx], c='lime', label='Out of Leverage', edgecolor='black', s=80)
plt.scatter(leverage[both_idx], std_residuals[both_idx], c='maroon', label='Suspected & Out of Leverage', s=90, edgecolor='black')

plt.xlabel('Hat')
plt.ylabel('Standardized residuals')
plt.title('Leverage vs Standardized Residuals (ANN-SEC)')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("leverage_residuals_sec.png", dpi=600, bbox_inches='tight')
plt.show()

# Calculate the number of points in each category
num_valid = np.sum(valid_idx)
num_suspect = np.sum(suspect_idx)
num_out_of_leverage = np.sum(leverage_idx)
num_both = np.sum(both_idx)

# Total points
total_points = len(X_train)

# Calculate the percentages
valid_percentage = (num_valid / total_points) * 100
suspect_percentage = (num_suspect / total_points) * 100
out_of_leverage_percentage = (num_out_of_leverage / total_points) * 100
both_percentage = (num_both / total_points) * 100

# Print results
print(f"Number of valid data points: {num_valid} ({valid_percentage:.2f}%)")
print(f"Number of suspected data points: {num_suspect} ({suspect_percentage:.2f}%)")
print(f"Number of out of leverage data points: {num_out_of_leverage} ({out_of_leverage_percentage:.2f}%)")
print(f"Number of suspected & out of leverage data points: {num_both} ({both_percentage:.2f}%)")

### **Purity**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers


# محاسبه استاندارد شده‌ی باقیمانده‌ها و leverage
y_true = df_label3
X_train = df.drop(columns=['Unnamed: 0'])


# model
n_features = X_train.shape[1]
# Best model
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)                       # single output
])

# Compile
ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_absolute_error')
# fit/predict/evaluate
ann_model1.fit(X_train, y_true, epochs=101, batch_size=32, verbose=0)


y_pred = ann_model1.predict(X_train)
y_pred = np.ravel(y_pred)          # (3540,)
y_true = np.ravel(y_true)          # (3540,)
residuals = y_true - y_pred
std_residuals = residuals / np.std(residuals)

# طراحی X با ستون ثابت
X_design = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
XT_X_inv = np.linalg.inv(X_design.T.dot(X_design))
H = X_design @ XT_X_inv @ X_design.T
leverage = np.diag(H)

# آستانه‌ها
n, p = X_train.shape
h_star = 3*(p+1)/n
resid_thresh = 3

# دسته‌بندی نقاط
valid_idx     = (np.abs(std_residuals) <= resid_thresh) & (leverage <= h_star)
suspect_idx   = (np.abs(std_residuals) >  resid_thresh) & (leverage <= h_star)
leverage_idx  = (leverage > h_star) & (np.abs(std_residuals) <= resid_thresh)
both_idx      = (leverage > h_star) & (np.abs(std_residuals) > resid_thresh)

# رسم نمودار
plt.figure(figsize=(7, 4))
plt.axhline(resid_thresh,  color='darkorange', linestyle='--', label='Upper suspected limit')
plt.axhline(-resid_thresh, color='darkorange', linestyle='--', label='Lower suspected limit')
plt.axvline(h_star,        color='purple', linestyle='--', label='Out of Leverage')

# نقاط
plt.scatter(leverage[valid_idx], std_residuals[valid_idx],   c='darkblue', label='Valid data', s=50)
plt.scatter(leverage[suspect_idx], std_residuals[suspect_idx], c='red', label='Suspected data', edgecolor='black', s=80)
plt.scatter(leverage[leverage_idx], std_residuals[leverage_idx], c='lime', label='Out of Leverage', edgecolor='black', s=80)
plt.scatter(leverage[both_idx], std_residuals[both_idx], c='maroon', label='Suspected & Out of Leverage', s=90, edgecolor='black')

plt.xlabel('Hat')
plt.ylabel('Standardized residuals')
plt.title('Leverage vs Standardized Residuals (ANN-Purity)')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("leverage_residuals_pur.png", dpi=600, bbox_inches='tight')
plt.show()

# Calculate the number of points in each category
num_valid = np.sum(valid_idx)
num_suspect = np.sum(suspect_idx)
num_out_of_leverage = np.sum(leverage_idx)
num_both = np.sum(both_idx)

# Total points
total_points = len(X_train)

# Calculate the percentages
valid_percentage = (num_valid / total_points) * 100
suspect_percentage = (num_suspect / total_points) * 100
out_of_leverage_percentage = (num_out_of_leverage / total_points) * 100
both_percentage = (num_both / total_points) * 100

# Print results
print(f"Number of valid data points: {num_valid} ({valid_percentage:.2f}%)")
print(f"Number of suspected data points: {num_suspect} ({suspect_percentage:.2f}%)")
print(f"Number of out of leverage data points: {num_out_of_leverage} ({out_of_leverage_percentage:.2f}%)")
print(f"Number of suspected & out of leverage data points: {num_both} ({both_percentage:.2f}%)")

## Surface plot

### **Recovery**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from mpl_toolkits.mplot3d import Axes3D  # This is needed for 3D plots

def surface_plot_2d(model, X_ref_df, feat_x, feat_y, ax, title="Surface plot", n_grid=60, fixed_strategy="median", cmap="viridis"):
    """
    model: trained model with predict(X)
    X_ref_df: pandas DataFrame of features (use train or total)
    feat_x, feat_y: two feature names to vary
    ax: Matplotlib axis to plot on (for subplot layout)
    fixed_strategy: "median" or "mean"
    cmap: Colormap for the surface plot
    """
    X_ref = X_ref_df.copy()

    # fixed point for other features
    if fixed_strategy == "median":
        base = X_ref.median(numeric_only=True)
    else:
        base = X_ref.mean(numeric_only=True)

    # grid ranges based on data
    x_vals = np.linspace(X_ref[feat_x].min(), X_ref[feat_x].max(), n_grid)
    y_vals = np.linspace(X_ref[feat_y].min(), X_ref[feat_y].max(), n_grid)
    XX, YY = np.meshgrid(x_vals, y_vals)

    # build grid dataframe for prediction
    grid = pd.DataFrame(np.tile(base.values, (n_grid*n_grid, 1)), columns=base.index)
    grid[feat_x] = XX.ravel()
    grid[feat_y] = YY.ravel()

    # ensure column order matches training
    grid = grid[X_ref_df.columns]

    ZZ = model.predict(grid).reshape(n_grid, n_grid)

    # --- Plot on provided 3D axis ---
    surface = ax.plot_surface(XX, YY, ZZ, linewidth=0, antialiased=True, alpha=0.9, cmap=cmap)
    ax.set_xlabel(feat_x)
    ax.set_ylabel(feat_y)
    ax.set_zlabel("Predicted")
    ax.set_title(title)

    # Add color bar for each surface plot
    fig.colorbar(surface, ax=ax, shrink=0.5, aspect=10)

# Prepare plot grid with 3 rows and 2 columns
fig, axes = plt.subplots(3, 2, figsize=(15, 18))

# Make sure to create 3D axes for each subplot
for row in range(3):
    for col in range(2):
        axes[row, col] = fig.add_subplot(3, 2, 2 * row + col + 1, projection="3d")  # Convert to 3D axis

# List of features pairs and titles for surface plots and different colormaps
plot_configs = [
    ("Fluegas Flow (kg/h)", "Amine type", "Recovery surface: flue gas flow vs amine type", "coolwarm"),
    ("Fluegas Flow (kg/h)", "Fluegas CO2 mol%", "Recovery surface: flue gas flow vs CO2 mol%", "coolwarm"),
    ("Fluegas Flow (kg/h)", "Lean Solvent Loading", "Recovery surface: flue gas flow vs loading", "coolwarm"),
    ("Fluegas CO2 mol%", "Amine type", "Recovery surface: CO2 mol% vs amine type", "coolwarm"),
    ("Fluegas CO2 mol%", "Lean Solvent Loading", "Recovery surface: CO2 mol% vs loading", "coolwarm"),
    ("Amine type", "Lean Solvent Loading", "Recovery surface: amine type vs loading", "coolwarm")
]

# Ensure the correct feature names and target values are loaded
X_train = df  # X_train should be your feature DataFrame
y_true = df_label1  # Ensure you have the correct target for the ANN

# Model setup (ensure it's already trained)
n_features = X_train.shape[1]  # Your features are in X_train
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Single output
])

ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mean_absolute_error')
ann_model1.fit(X_train, y_true, epochs=101, batch_size=32, verbose=0)

# Generate surface plots for each pair of features in the configuration list
for i, (feat_x, feat_y, title, cmap) in enumerate(plot_configs):
    row, col = divmod(i, 2)
    surface_plot_2d(ann_model1, X_train, feat_x, feat_y, axes[row, col], title=title, n_grid=100, cmap=cmap)

# Adjust layout and save as SVG
plt.tight_layout()
plt.savefig("surface_plots_rec.svg", format="svg")
plt.show()


### **SEC**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from mpl_toolkits.mplot3d import Axes3D  # This is needed for 3D plots

def surface_plot_2d(model, X_ref_df, feat_x, feat_y, ax, title="Surface plot", n_grid=60, fixed_strategy="median", cmap="magma"):
    """
    model: trained model with predict(X)
    X_ref_df: pandas DataFrame of features (use train or total)
    feat_x, feat_y: two feature names to vary
    ax: Matplotlib axis to plot on (for subplot layout)
    fixed_strategy: "median" or "mean"
    cmap: Colormap for the surface plot
    """
    X_ref = X_ref_df.copy()

    # fixed point for other features
    if fixed_strategy == "median":
        base = X_ref.median(numeric_only=True)
    else:
        base = X_ref.mean(numeric_only=True)

    # grid ranges based on data
    x_vals = np.linspace(X_ref[feat_x].min(), X_ref[feat_x].max(), n_grid)
    y_vals = np.linspace(X_ref[feat_y].min(), X_ref[feat_y].max(), n_grid)
    XX, YY = np.meshgrid(x_vals, y_vals)

    # build grid dataframe for prediction
    grid = pd.DataFrame(np.tile(base.values, (n_grid*n_grid, 1)), columns=base.index)
    grid[feat_x] = XX.ravel()
    grid[feat_y] = YY.ravel()

    # ensure column order matches training
    grid = grid[X_ref.columns]

    ZZ = model.predict(grid).reshape(n_grid, n_grid)

    # --- Plot on provided 3D axis ---
    surface = ax.plot_surface(XX, YY, ZZ, linewidth=0, antialiased=True, alpha=0.9, cmap=cmap)
    ax.set_xlabel(feat_x)
    ax.set_ylabel(feat_y)
    ax.set_zlabel("Predicted", labelpad=10)  # Adjust label padding to avoid overlap
    ax.set_title(title)

    # Hide the SEC axis values (i.e., don't display any ticks on the z-axis)
    ax.zaxis.set_ticks([])  # Hides the ticks on the z-axis

    # Add color bar for each surface plot
    fig.colorbar(surface, ax=ax, shrink=0.5, aspect=10)

# Prepare plot grid with 5 rows and 2 columns
fig, axes = plt.subplots(5, 2, figsize=(15, 25))  # Increase figure size to fit 10 subplots

# Make sure to create 3D axes for each subplot
for row in range(5):
    for col in range(2):
        axes[row, col] = fig.add_subplot(5, 2, 2 * row + col + 1, projection="3d")  # Convert to 3D axis

# List of features pairs and titles for surface plots and different colormaps
plot_configs = [
    ("Fluegas Flow (kg/h)", "Amine type", "SEC surface: flue gas flow vs amine type", "magma"),
    ("Fluegas Flow (kg/h)", "Amine Conc (wt%)", "SEC surface: flue gas flow vs amine concentration", "magma"),
    ("Fluegas Flow (kg/h)", "Fluegas CO2 mol%", "SEC surface: flue gas flow vs CO2 mol%", "magma"),
    ("Fluegas Flow (kg/h)", "Reboiler Duty (MW)", "SEC surface: flue gas flow vs reboiler duty", "magma"),
    ("Fluegas CO2 mol%", "Amine type", "SEC surface: CO2 mol% vs amine type", "magma"),
    ("Fluegas CO2 mol%", "Reboiler Duty (MW)", "SEC surface: CO2 mol% vs reboiler duty", "magma"),
    ("Fluegas CO2 mol%", "Amine Conc (wt%)", "SEC surface: CO2 mol% vs amine concentration", "magma"),
    ("Amine type", "Reboiler Duty (MW)", "SEC surface: amine type vs reboiler duty", "magma"),
    ("Amine type", "Amine Conc (wt%)", "SEC surface: amine type vs amine concentration", "magma"),
    ("Reboiler Duty (MW)", "Amine Conc (wt%)", "SEC surface: reboiler duty vs amine concentration", "magma")
]

# Ensure the correct feature names and target values are loaded
X_train = df  # X_train should be your feature DataFrame
y_true = df_label2  # Ensure you have the correct target for the ANN

# Model setup (ensure it's already trained)
n_features = X_train.shape[1]  # Your features are in X_train
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(120, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)  # Single output
])

ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')
ann_model1.fit(X_train, y_true, epochs=101, batch_size=32, verbose=0)

# Generate surface plots for each pair of features in the configuration list
for i, (feat_x, feat_y, title, cmap) in enumerate(plot_configs):
    row, col = divmod(i, 2)
    surface_plot_2d(ann_model1, X_train, feat_x, feat_y, axes[row, col], title=title, n_grid=100, cmap=cmap)

# Adjust layout and save as SVG
plt.tight_layout()
plt.savefig("surface_plots_sec_without_ticks.svg", format="svg")
plt.show()


### **Purity**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from mpl_toolkits.mplot3d import Axes3D  # This is needed for 3D plots

def surface_plot_2d(model, X_ref_df, feat_x, feat_y, ax, title="Surface plot", n_grid=60, fixed_strategy="median", cmap="viridis"):
    """
    model: trained model with predict(X)
    X_ref_df: pandas DataFrame of features (use train or total)
    feat_x, feat_y: two feature names to vary
    ax: Matplotlib axis to plot on (for subplot layout)
    fixed_strategy: "median" or "mean"
    cmap: Colormap for the surface plot
    """
    X_ref = X_ref_df.copy()

    # fixed point for other features
    if fixed_strategy == "median":
        base = X_ref.median(numeric_only=True)
    else:
        base = X_ref.mean(numeric_only=True)

    # grid ranges based on data
    x_vals = np.linspace(X_ref[feat_x].min(), X_ref[feat_x].max(), n_grid)
    y_vals = np.linspace(X_ref[feat_y].min(), X_ref[feat_y].max(), n_grid)
    XX, YY = np.meshgrid(x_vals, y_vals)

    # build grid dataframe for prediction
    grid = pd.DataFrame(np.tile(base.values, (n_grid*n_grid, 1)), columns=base.index)
    grid[feat_x] = XX.ravel()
    grid[feat_y] = YY.ravel()

    # ensure column order matches training
    grid = grid[X_ref_df.columns]

    ZZ = model.predict(grid).reshape(n_grid, n_grid)

    # --- Plot on provided 3D axis ---
    surface = ax.plot_surface(XX, YY, ZZ, linewidth=0, antialiased=True, alpha=0.9, cmap=cmap)
    ax.set_xlabel(feat_x)
    ax.set_ylabel(feat_y)
    ax.set_zlabel("Predicted")
    ax.set_title(title)

    # Add color bar for each surface plot
    fig.colorbar(surface, ax=ax, shrink=0.5, aspect=10)

# Prepare plot grid with 3 rows and 2 columns
fig, axes = plt.subplots(3, 2, figsize=(15, 18))

# Make sure to create 3D axes for each subplot
for row in range(3):
    for col in range(2):
        axes[row, col] = fig.add_subplot(3, 2, 2 * row + col + 1, projection="3d")  # Convert to 3D axis

# List of features pairs and titles for surface plots and different colormaps
plot_configs = [
    ("Fluegas Flow (kg/h)", "Amine type", "Purity surface: flue gas flow vs amine type", "cividis"),
    ("Fluegas Flow (kg/h)", "Fluegas CO2 mol%", "Purity surface: flue gas flow vs CO2 mol%", "cividis"),
    ("Fluegas Flow (kg/h)", "Lean Solvent Loading", "Purity surface: flue gas flow vs loading", "cividis"),
    ("Fluegas CO2 mol%", "Amine type", "Purity surface: CO2 mol% vs amine type", "cividis"),
    ("Fluegas CO2 mol%", "Lean Solvent Loading", "Purity surface: CO2 mol% vs loading", "cividis"),
    ("Amine type", "Lean Solvent Loading", "Purity surface: amine type vs loading", "cividis")
]

# Ensure the correct feature names and target values are loaded
X_train = df  # X_train should be your feature DataFrame
y_true = df_label3  # Ensure you have the correct target for the ANN

# Model setup (ensure it's already trained)
n_features = X_train.shape[1]  # Your features are in X_train
ann_model1 = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(72, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Single output
])

ann_model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mean_absolute_error')
ann_model1.fit(X_train, y_true, epochs=101, batch_size=32, verbose=0)

# Generate surface plots for each pair of features in the configuration list
for i, (feat_x, feat_y, title, cmap) in enumerate(plot_configs):
    row, col = divmod(i, 2)
    surface_plot_2d(ann_model1, X_train, feat_x, feat_y, axes[row, col], title=title, n_grid=100, cmap=cmap)

# Adjust layout and save as SVG
plt.tight_layout()
plt.savefig("surface_plots_pur.svg", format="svg")
plt.show()
